In [4063]:

# SEAG qualification period is 22 Oct 2024 to 5 Sept 2025

#1. Segregate into Male and Femal 
#2. For each gender perform the following: 
#a. Sort data by mapped eent, then perf scalar (higher the better)
#b. Identify tiers based on performance - Tier 1 is meets bronze medal mark for SEAG, Tier 2 is 2% and Tier 3 is 3.5%
#c. Check - if athlete met bronze or 2%/3.5% then delta_benchmark is zero or +, delta2% is + and delta 3.5% is +
#d. Top ranked athletes for each event are chosen. Max number of athletes for each event is 3, except for 100m/400m which is 6
#    This includes athletes on spex scholarship and potential
#e. The max for each tier is 2. Lower ranked athletes move down one tier.
#3. If athlete qualifies for more than one event the higher tier event is given
#4. Jump and throws junior program to be solved separately

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [4064]:
# Import usual modules
import pandas as pd
import csv
import math
import os
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import openpyxl
import datetime
from scipy.stats import lognorm
import re
import string
from bs4 import BeautifulSoup
import requests
import unicodedata # for removing accented characters
import datetime
import icecream as ic
import dateutil.parser as parser 
import datacompy
from itertools import permutations



from google.cloud import storage



In [4065]:
# BACKUP DATABASE
# Extract timed event records

import pandas_gbq
from google.oauth2 import service_account

credentials = service_account.Credentials.from_service_account_file(
    '/Users/veesheenyuen/Desktop/DataScience/Keys/saa-analytics-7c8937b70609.json',
    
    
)

sql1="""
SELECT * FROM `saa-analytics.results.athlete_results_prod` 
"""

backup = pandas_gbq.read_gbq(sql1, project_id="saa-analytics", credentials=credentials)

os.chdir('/Users/veesheenyuen/Desktop/DataScience/SAA/Backup DB/')


backup.to_csv('Backup_DB_June25.csv', sep=',', encoding='utf-8-sig', index=False)


Downloading: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████|


In [4066]:
# PRODUCTION ENVIRONMENT
# Extract timed event records

import pandas_gbq
from google.oauth2 import service_account

credentials = service_account.Credentials.from_service_account_file(
    '/Users/veesheenyuen/Desktop/DataScience/Keys/saa-analytics-7c8937b70609.json',
    
    
)

sql1="""
SELECT NAME, RESULT, TEAM, AGE, RANK AS COMPETITION_RANK, DIVISION, EVENT, DISTANCE, EVENT_CLASS, UNIQUE_ID, DOB, NATIONALITY, WIND, CATEGORY_EVENT, GENDER, COMPETITION, DATE, YEAR, REGION, TIMESTAMP
FROM `saa-analytics.results.athlete_results_prod` 
WHERE RESULT!='NM' AND RESULT!='-' AND RESULT!='DNS' AND RESULT!='DNF' AND RESULT!='DNQ' AND RESULT!='DQ' AND RESULT IS NOT NULL

"""

competitors = pandas_gbq.read_gbq(sql1, project_id="saa-analytics", credentials=credentials)




Downloading: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████|


In [4067]:
competitors

,NAME,RESULT,TEAM,AGE,COMPETITION_RANK,DIVISION,EVENT,DISTANCE,EVENT_CLASS,UNIQUE_ID,DOB,NATIONALITY,WIND,CATEGORY_EVENT,GENDER,COMPETITION,DATE,YEAR,REGION,TIMESTAMP
0,Zubin Percy Muncherji,01:54.0,None,None,2.0,None,800m,None,None,None,23-Jun-96,SGP,None,Mid,Male,Taiwan Athletics Open,7 - 8 June,2025,International,2025-06-11 16:38:50.923703
1,Nicole Low Sui Xuan,18:34.34,None,None,6.0,None,5000m,None,None,None,None,SGP,None,Long,Female,Taiwan Athletics Open,7 - 8 June,2025,International,2025-06-11 16:38:50.923703
2,Gabriel Lee,15.15,None,None,7.0,None,Triple Jump,None,None,None,23-Feb-03,SGP,None,Jump,Male,Taiwan Athletics Open,7 - 8 June,2025,International,2025-06-11 16:38:50.923703
3,Esther Tay Shee Wei,3.2,None,None,10.0,None,Pole Vault,None,None,None,None,SGP,None,Jump,Female,Taiwan Athletics Open,7 - 8 June,2025,International,2025-06-11 16:38:50.923703
4,Romaine Soh,02:16.90,None,None,5.0,None,800m,None,None,None,26-Nov-94,SGP,None,Mid,Female,Taiwan Athletics Open,7 - 8 June,2025,International,2025-06-11 16:38:50.923703
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31154,St. Andrew's Junior College,03:43.06,SAJC,None,5.0,A,4x400m Relay,None,None,None,None,None,None,Relay,Male,National School Games,28-04,2025,Local,2025-06-06 15:35:30.807106
31155,Catholic Junior College,03:43.67,CJC,None,6.0,A,4x400m Relay,None,None,None,None,None,None,Relay,Male,National School Games,28-04,2025,Local,2025-06-06 15:35:30.807106
31156,Yishun Innova Junior College,03:45.03,YIJC,None,7.0,A,4x400m Relay,None,None,None,None,None,None,Relay,Male,National School Games,28-04,2025,Local,2025-06-06 15:35:30.807106
31157,National Junior College,03:47.40,NJC,None,8.0,A,4x400m Relay,None,None,None,None,None,None,Relay,Male,National School Games,28-04,2025,Local,2025-06-06 15:35:30.807106


In [4068]:
os.chdir('/Users/veesheenyuen/Desktop/DataScience/SAA/Asian Youth/')


competitors.to_csv('athletes_new_format_prod.csv', sep=',', encoding='utf-8-sig', index=False)

In [4069]:
competitors

,NAME,RESULT,TEAM,AGE,COMPETITION_RANK,DIVISION,EVENT,DISTANCE,EVENT_CLASS,UNIQUE_ID,DOB,NATIONALITY,WIND,CATEGORY_EVENT,GENDER,COMPETITION,DATE,YEAR,REGION,TIMESTAMP
0,Zubin Percy Muncherji,01:54.0,None,None,2.0,None,800m,None,None,None,23-Jun-96,SGP,None,Mid,Male,Taiwan Athletics Open,7 - 8 June,2025,International,2025-06-11 16:38:50.923703
1,Nicole Low Sui Xuan,18:34.34,None,None,6.0,None,5000m,None,None,None,None,SGP,None,Long,Female,Taiwan Athletics Open,7 - 8 June,2025,International,2025-06-11 16:38:50.923703
2,Gabriel Lee,15.15,None,None,7.0,None,Triple Jump,None,None,None,23-Feb-03,SGP,None,Jump,Male,Taiwan Athletics Open,7 - 8 June,2025,International,2025-06-11 16:38:50.923703
3,Esther Tay Shee Wei,3.2,None,None,10.0,None,Pole Vault,None,None,None,None,SGP,None,Jump,Female,Taiwan Athletics Open,7 - 8 June,2025,International,2025-06-11 16:38:50.923703
4,Romaine Soh,02:16.90,None,None,5.0,None,800m,None,None,None,26-Nov-94,SGP,None,Mid,Female,Taiwan Athletics Open,7 - 8 June,2025,International,2025-06-11 16:38:50.923703
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31154,St. Andrew's Junior College,03:43.06,SAJC,None,5.0,A,4x400m Relay,None,None,None,None,None,None,Relay,Male,National School Games,28-04,2025,Local,2025-06-06 15:35:30.807106
31155,Catholic Junior College,03:43.67,CJC,None,6.0,A,4x400m Relay,None,None,None,None,None,None,Relay,Male,National School Games,28-04,2025,Local,2025-06-06 15:35:30.807106
31156,Yishun Innova Junior College,03:45.03,YIJC,None,7.0,A,4x400m Relay,None,None,None,None,None,None,Relay,Male,National School Games,28-04,2025,Local,2025-06-06 15:35:30.807106
31157,National Junior College,03:47.40,NJC,None,8.0,A,4x400m Relay,None,None,None,None,None,None,Relay,Male,National School Games,28-04,2025,Local,2025-06-06 15:35:30.807106


In [4070]:
competitors[competitors['NAME']=='SOH RUI YONG GUILLAUME']

,NAME,RESULT,TEAM,AGE,COMPETITION_RANK,DIVISION,EVENT,DISTANCE,EVENT_CLASS,UNIQUE_ID,DOB,NATIONALITY,WIND,CATEGORY_EVENT,GENDER,COMPETITION,DATE,YEAR,REGION,TIMESTAMP
2189,SOH RUI YONG GUILLAUME,04:59.3,INDIVIDUAL,33,12,None,1 mile,None,None,None,1991,SGP,-,Mid,Male,Vitaly West Mininster Mile 5K (UK),22-Sep,2024,International,None
2231,SOH RUI YONG GUILLAUME,08:35.8,INDIVIDUAL,33,None,None,3000m,None,None,None,1991,SGP,-,Long,Male,Hercules Wimbledon Athletics Club 3000m,14-Jun,2024,International,None
3826,SOH RUI YONG GUILLAUME,31:11.4,INDIVIDUAL,33,133,None,10000m,None,None,None,1991,SGP,-,Long,Male,"Tokai University Long Distance Challenge 10,0...",25-Nov,2024,International,None
22084,SOH RUI YONG GUILLAUME,15:07.7,INDIVIDUAL,33,46,None,5000m,None,None,None,1991,SGP,-,Long,Male,Friday Night Under the Lights 5K (UK),13-Sep,2024,International,None


In [4071]:
# Create mm/dd/yy column for date of event

import datetime

#s=247.779

#datetime.datetime.fromtimestamp(s).strftime('%M:%S.%f')

for i in range(len(competitors)):
        
    rowIndex = competitors.index[i]

    date = competitors.loc[rowIndex,'DATE']
    year = competitors.loc[rowIndex,'YEAR']    
    
    if 'to' in date or ' - ' in date:
        
        if re.search('to|\s\-\s\d\s|\s\-\d\d', date):  # e.g. 03-04
              
            pos = re.search('to|\s\-\s\d', date)
            # Splice string to day and month

            split_pos_start=pos.start()+3
            


            final_date = date[split_pos_start:] # left string post splicing

            print(i, pos, date, final_date)
            final_year = year[2:]

            event_date = final_date + '/' + final_year

            print('old code', date, final_date, event_date)

            competitors.loc[rowIndex, 'event_date'] = event_date

        elif re.search('(\-\s\d\w)|(\-\s\d\d\w)', date):  # e.g. 18 - 19 January
                        
            pos = re.search('\-', date)  # from '-' onwards only
            # Splice string to day and month

            split_pos_start=pos.start()+2
            

            final_date = date[split_pos_start:] # left string post splicing

            
            final_year = year[2:]

            event_date = final_date + ' ' + final_year


            competitors.loc[rowIndex, 'event_date'] = event_date
            
        
    elif re.search('\w\-\w', date):
        
        if competitors.loc[rowIndex, 'COMPETITION'] == "National School Games":
            
            if competitors.loc[rowIndex, 'YEAR'] == '2024':
        
                event_date = '04'+'/'+date[1:3] + '/' + year[2:]  # reverse order from dd/mm to mm/dd. 04 because event was in April 24 only
            
       #         print('NSG 2024', event_date)
        
                competitors.loc[rowIndex, 'event_date'] = event_date
            
            elif competitors.loc[rowIndex, 'YEAR'] == '2025':
                
                event_date = date + '-' +year[2:]
                
        #        print('NSG2025', event_date)
                
                competitors.loc[rowIndex, 'event_date'] = event_date
                
        elif re.search('\d\-\d',  date):        #10-13 April
            
            print('HERE', i, date)

            rpos = re.search('\-', date)
            string = date[rpos.end():]
            
            print('extracted date', string)
            
            event_date = string + ' ' + year
            
            print('event date', event_date)
            
            competitors.loc[rowIndex, 'event_date'] = event_date

            
        
        else:
            
            event_date = date + '-' + year[2:]
            
            competitors.loc[rowIndex, 'event_date'] = event_date

        
        

        
competitors['event_date'] = competitors['event_date'].astype(str)
competitors['event_date'] = competitors['event_date'].str.replace('\xa0', ' ', regex=True)
competitors['event_date'] = competitors['event_date'].str.replace('[\x00-\x1f\x7f-\x9f]', '', regex=True)
competitors['event_date'] = competitors['event_date'].str.replace('\r', ' ', regex=True)
competitors['event_date'] = competitors['event_date'].str.replace('\n', ' ', regex=True)
competitors['event_date'] = competitors['event_date'].str.strip()
       
    

0 <re.Match object; span=(1, 5), match=' - 8'> 7 - 8 June 8 June
old code 7 - 8 June 8 June 8 June/25
1 <re.Match object; span=(1, 5), match=' - 8'> 7 - 8 June 8 June
old code 7 - 8 June 8 June 8 June/25
2 <re.Match object; span=(1, 5), match=' - 8'> 7 - 8 June 8 June
old code 7 - 8 June 8 June 8 June/25
3 <re.Match object; span=(1, 5), match=' - 8'> 7 - 8 June 8 June
old code 7 - 8 June 8 June 8 June/25
4 <re.Match object; span=(1, 5), match=' - 8'> 7 - 8 June 8 June
old code 7 - 8 June 8 June 8 June/25
5 <re.Match object; span=(1, 5), match=' - 8'> 7 - 8 June 8 June
old code 7 - 8 June 8 June 8 June/25
6 <re.Match object; span=(1, 5), match=' - 8'> 7 - 8 June 8 June
old code 7 - 8 June 8 June 8 June/25
7 <re.Match object; span=(1, 5), match=' - 8'> 7 - 8 June 8 June
old code 7 - 8 June 8 June 8 June/25
8 <re.Match object; span=(1, 5), match=' - 8'> 7 - 8 June 8 June
old code 7 - 8 June 8 June 8 June/25
11 <re.Match object; span=(1, 5), match=' - 8'> 7 - 8 June 8 June
old code 7 - 8 J

1567 <re.Match object; span=(5, 7), match='to'> 3/13 to 3/30 3/30
old code 3/13 to 3/30 3/30 3/30/25
1568 <re.Match object; span=(5, 7), match='to'> 3/13 to 3/30 3/30
old code 3/13 to 3/30 3/30 3/30/25
1569 <re.Match object; span=(5, 7), match='to'> 3/13 to 3/30 3/30
old code 3/13 to 3/30 3/30 3/30/25
1570 <re.Match object; span=(5, 7), match='to'> 3/23 to 3/24 3/24
old code 3/23 to 3/24 3/24 3/24/24
1571 <re.Match object; span=(4, 6), match='to'> 3/9 to 3/9 3/9
old code 3/9 to 3/9 3/9 3/9/24
1572 <re.Match object; span=(4, 6), match='to'> 3/9 to 3/9 3/9
old code 3/9 to 3/9 3/9 3/9/24
1573 <re.Match object; span=(4, 6), match='to'> 3/9 to 3/9 3/9
old code 3/9 to 3/9 3/9 3/9/24
1574 <re.Match object; span=(5, 7), match='to'> 4/24 to 4/25 4/25
old code 4/24 to 4/25 4/25 4/25/25
1575 <re.Match object; span=(5, 7), match='to'> 4/24 to 4/25 4/25
old code 4/24 to 4/25 4/25 4/25/25
1576 <re.Match object; span=(5, 7), match='to'> 4/24 to 4/25 4/25
old code 4/24 to 4/25 4/25 4/25/25
1577 <re.Ma

3211 <re.Match object; span=(4, 6), match='to'> 2/9 to 2/16 2/16
old code 2/9 to 2/16 2/16 2/16/25
3212 <re.Match object; span=(4, 6), match='to'> 2/9 to 2/16 2/16
old code 2/9 to 2/16 2/16 2/16/25
3213 <re.Match object; span=(5, 7), match='to'> 3/13 to 3/17 3/17
old code 3/13 to 3/17 3/17 3/17/24
3214 <re.Match object; span=(5, 7), match='to'> 3/13 to 3/17 3/17
old code 3/13 to 3/17 3/17 3/17/24
3215 <re.Match object; span=(5, 7), match='to'> 3/13 to 3/17 3/17
old code 3/13 to 3/17 3/17 3/17/24
3216 <re.Match object; span=(5, 7), match='to'> 3/13 to 3/17 3/17
old code 3/13 to 3/17 3/17 3/17/24
3217 <re.Match object; span=(5, 7), match='to'> 3/13 to 3/17 3/17
old code 3/13 to 3/17 3/17 3/17/24
3218 <re.Match object; span=(5, 7), match='to'> 3/13 to 3/30 3/30
old code 3/13 to 3/30 3/30 3/30/25
3219 <re.Match object; span=(5, 7), match='to'> 3/13 to 3/30 3/30
old code 3/13 to 3/30 3/30 3/30/25
3220 <re.Match object; span=(5, 7), match='to'> 3/13 to 3/30 3/30
old code 3/13 to 3/30 3/30 3/

4819 <re.Match object; span=(5, 7), match='to'> 7/27 to 7/28 7/28
old code 7/27 to 7/28 7/28 7/28/24
4820 <re.Match object; span=(5, 7), match='to'> 7/27 to 7/28 7/28
old code 7/27 to 7/28 7/28 7/28/24
4821 <re.Match object; span=(5, 7), match='to'> 7/27 to 7/28 7/28
old code 7/27 to 7/28 7/28 7/28/24
4822 <re.Match object; span=(5, 7), match='to'> 7/27 to 7/28 7/28
old code 7/27 to 7/28 7/28 7/28/24
4823 <re.Match object; span=(5, 7), match='to'> 7/27 to 7/28 7/28
old code 7/27 to 7/28 7/28 7/28/24
4824 <re.Match object; span=(5, 7), match='to'> 7/27 to 7/28 7/28
old code 7/27 to 7/28 7/28 7/28/24
4825 <re.Match object; span=(5, 7), match='to'> 7/27 to 7/28 7/28
old code 7/27 to 7/28 7/28 7/28/24
4826 <re.Match object; span=(5, 7), match='to'> 7/27 to 7/28 7/28
old code 7/27 to 7/28 7/28 7/28/24
4827 <re.Match object; span=(5, 7), match='to'> 7/27 to 7/28 7/28
old code 7/27 to 7/28 7/28 7/28/24
4828 <re.Match object; span=(5, 7), match='to'> 1/10 to 1/14 1/14
old code 1/10 to 1/14 1/1

6476 <re.Match object; span=(1, 5), match=' - 2'> 1 - 2 June 2 June
old code 1 - 2 June 2 June 2 June/24
6509 <re.Match object; span=(5, 7), match='to'> 1/27 to 1/28 1/28
old code 1/27 to 1/28 1/28 1/28/24
6510 <re.Match object; span=(5, 7), match='to'> 1/27 to 1/28 1/28
old code 1/27 to 1/28 1/28 1/28/24
6511 <re.Match object; span=(5, 7), match='to'> 1/27 to 1/28 1/28
old code 1/27 to 1/28 1/28 1/28/24
6512 <re.Match object; span=(5, 7), match='to'> 1/27 to 1/28 1/28
old code 1/27 to 1/28 1/28 1/28/24
6513 <re.Match object; span=(5, 7), match='to'> 1/27 to 1/28 1/28
old code 1/27 to 1/28 1/28 1/28/24
6514 <re.Match object; span=(5, 7), match='to'> 1/27 to 1/28 1/28
old code 1/27 to 1/28 1/28 1/28/24
6515 <re.Match object; span=(4, 6), match='to'> 1/8 to 1/12 1/12
old code 1/8 to 1/12 1/12 1/12/25
6516 <re.Match object; span=(4, 6), match='to'> 1/8 to 1/12 1/12
old code 1/8 to 1/12 1/12 1/12/25
6517 <re.Match object; span=(4, 6), match='to'> 1/8 to 1/12 1/12
old code 1/8 to 1/12 1/12 

...

17966 <re.Match object; span=(4, 6), match='to'> 2/9 to 2/16 2/16
old code 2/9 to 2/16 2/16 2/16/25
17967 <re.Match object; span=(5, 7), match='to'> 3/13 to 3/30 3/30
old code 3/13 to 3/30 3/30 3/30/25
17968 <re.Match object; span=(5, 7), match='to'> 3/13 to 3/30 3/30
old code 3/13 to 3/30 3/30 3/30/25
17969 <re.Match object; span=(5, 7), match='to'> 3/13 to 3/30 3/30
old code 3/13 to 3/30 3/30 3/30/25
17970 <re.Match object; span=(5, 7), match='to'> 3/23 to 3/24 3/24
old code 3/23 to 3/24 3/24 3/24/24
17971 <re.Match object; span=(4, 6), match='to'> 3/9 to 3/9 3/9
old code 3/9 to 3/9 3/9 3/9/24
17972 <re.Match object; span=(5, 7), match='to'> 6/22 to 6/22 6/22
old code 6/22 to 6/22 6/22 6/22/24
17973 <re.Match object; span=(5, 7), match='to'> 1/27 to 1/28 1/28
old code 1/27 to 1/28 1/28 1/28/24
17974 <re.Match object; span=(5, 7), match='to'> 1/27 to 1/28 1/28
old code 1/27 to 1/28 1/28 1/28/24
17975 <re.Match object; span=(5, 7), match='to'> 1/27 to 1/28 1/28
old code 1/27 to 1/28 1/

19632 <re.Match object; span=(5, 7), match='to'> 3/23 to 3/24 3/24
old code 3/23 to 3/24 3/24 3/24/24
19633 <re.Match object; span=(5, 7), match='to'> 3/23 to 3/24 3/24
old code 3/23 to 3/24 3/24 3/24/24
19634 <re.Match object; span=(5, 7), match='to'> 3/23 to 3/24 3/24
old code 3/23 to 3/24 3/24 3/24/24
19635 <re.Match object; span=(5, 7), match='to'> 3/23 to 3/24 3/24
old code 3/23 to 3/24 3/24 3/24/24
19636 <re.Match object; span=(4, 6), match='to'> 3/9 to 3/9 3/9
old code 3/9 to 3/9 3/9 3/9/24
19637 <re.Match object; span=(4, 6), match='to'> 3/9 to 3/9 3/9
old code 3/9 to 3/9 3/9 3/9/24
19638 <re.Match object; span=(4, 6), match='to'> 3/9 to 3/9 3/9
old code 3/9 to 3/9 3/9 3/9/24
19639 <re.Match object; span=(5, 7), match='to'> 4/18 to 4/19 4/19
old code 4/18 to 4/19 4/19 4/19/24
19640 <re.Match object; span=(5, 7), match='to'> 4/18 to 4/19 4/19
old code 4/18 to 4/19 4/19 4/19/24
19641 <re.Match object; span=(5, 7), match='to'> 1/27 to 1/28 1/28
old code 1/27 to 1/28 1/28 1/28/24
1

21336 <re.Match object; span=(5, 7), match='to'> 2/23 to 3/2 3/2
old code 2/23 to 3/2 3/2 3/2/25
21337 <re.Match object; span=(5, 7), match='to'> 2/23 to 3/2 3/2
old code 2/23 to 3/2 3/2 3/2/25
21338 <re.Match object; span=(5, 7), match='to'> 2/23 to 3/2 3/2
old code 2/23 to 3/2 3/2 3/2/25
21339 <re.Match object; span=(5, 7), match='to'> 2/23 to 3/2 3/2
old code 2/23 to 3/2 3/2 3/2/25
21340 <re.Match object; span=(5, 7), match='to'> 2/23 to 3/2 3/2
old code 2/23 to 3/2 3/2 3/2/25
21341 <re.Match object; span=(5, 7), match='to'> 2/25 to 3/3 3/3
old code 2/25 to 3/3 3/3 3/3/24
21342 <re.Match object; span=(5, 7), match='to'> 2/25 to 3/3 3/3
old code 2/25 to 3/3 3/3 3/3/24
21343 <re.Match object; span=(5, 7), match='to'> 2/25 to 3/3 3/3
old code 2/25 to 3/3 3/3 3/3/24
21344 <re.Match object; span=(5, 7), match='to'> 2/25 to 3/3 3/3
old code 2/25 to 3/3 3/3 3/3/24
21345 <re.Match object; span=(5, 7), match='to'> 2/25 to 3/3 3/3
old code 2/25 to 3/3 3/3 3/3/24
21346 <re.Match object; span=(

23113 <re.Match object; span=(5, 7), match='to'> 1/10 to 1/14 1/14
old code 1/10 to 1/14 1/14 1/14/24
23114 <re.Match object; span=(5, 7), match='to'> 1/10 to 1/14 1/14
old code 1/10 to 1/14 1/14 1/14/24
23115 <re.Match object; span=(5, 7), match='to'> 1/10 to 1/14 1/14
old code 1/10 to 1/14 1/14 1/14/24
23116 <re.Match object; span=(5, 7), match='to'> 1/27 to 1/28 1/28
old code 1/27 to 1/28 1/28 1/28/24
23117 <re.Match object; span=(5, 7), match='to'> 1/27 to 1/28 1/28
old code 1/27 to 1/28 1/28 1/28/24
23118 <re.Match object; span=(5, 7), match='to'> 1/27 to 1/28 1/28
old code 1/27 to 1/28 1/28 1/28/24
23119 <re.Match object; span=(5, 7), match='to'> 1/27 to 1/28 1/28
old code 1/27 to 1/28 1/28 1/28/24
23120 <re.Match object; span=(5, 7), match='to'> 1/27 to 1/28 1/28
old code 1/27 to 1/28 1/28 1/28/24
23121 <re.Match object; span=(5, 7), match='to'> 1/27 to 1/28 1/28
old code 1/27 to 1/28 1/28 1/28/24
23122 <re.Match object; span=(5, 7), match='to'> 1/27 to 1/28 1/28
old code 1/27 t

25176 <re.Match object; span=(5, 7), match='to'> 1/27 to 1/28 1/28
old code 1/27 to 1/28 1/28 1/28/24
25177 <re.Match object; span=(5, 7), match='to'> 1/27 to 1/28 1/28
old code 1/27 to 1/28 1/28 1/28/24
25178 <re.Match object; span=(5, 7), match='to'> 1/27 to 1/28 1/28
old code 1/27 to 1/28 1/28 1/28/24
25179 <re.Match object; span=(5, 7), match='to'> 1/27 to 1/28 1/28
old code 1/27 to 1/28 1/28 1/28/24
25180 <re.Match object; span=(5, 7), match='to'> 1/27 to 1/28 1/28
old code 1/27 to 1/28 1/28 1/28/24
25181 <re.Match object; span=(5, 7), match='to'> 1/27 to 1/28 1/28
old code 1/27 to 1/28 1/28 1/28/24
25182 <re.Match object; span=(5, 7), match='to'> 1/27 to 1/28 1/28
old code 1/27 to 1/28 1/28 1/28/24
25183 <re.Match object; span=(5, 7), match='to'> 1/27 to 1/28 1/28
old code 1/27 to 1/28 1/28 1/28/24
25184 <re.Match object; span=(5, 7), match='to'> 1/27 to 1/28 1/28
old code 1/27 to 1/28 1/28 1/28/24
25185 <re.Match object; span=(5, 7), match='to'> 1/27 to 1/28 1/28
old code 1/27 t

In [4072]:
competitors

,NAME,RESULT,TEAM,AGE,COMPETITION_RANK,DIVISION,EVENT,DISTANCE,EVENT_CLASS,UNIQUE_ID,...,NATIONALITY,WIND,CATEGORY_EVENT,GENDER,COMPETITION,DATE,YEAR,REGION,TIMESTAMP,event_date
0,Zubin Percy Muncherji,01:54.0,None,None,2.0,None,800m,None,None,None,...,SGP,None,Mid,Male,Taiwan Athletics Open,7 - 8 June,2025,International,2025-06-11 16:38:50.923703,8 June/25
1,Nicole Low Sui Xuan,18:34.34,None,None,6.0,None,5000m,None,None,None,...,SGP,None,Long,Female,Taiwan Athletics Open,7 - 8 June,2025,International,2025-06-11 16:38:50.923703,8 June/25
2,Gabriel Lee,15.15,None,None,7.0,None,Triple Jump,None,None,None,...,SGP,None,Jump,Male,Taiwan Athletics Open,7 - 8 June,2025,International,2025-06-11 16:38:50.923703,8 June/25
3,Esther Tay Shee Wei,3.2,None,None,10.0,None,Pole Vault,None,None,None,...,SGP,None,Jump,Female,Taiwan Athletics Open,7 - 8 June,2025,International,2025-06-11 16:38:50.923703,8 June/25
4,Romaine Soh,02:16.90,None,None,5.0,None,800m,None,None,None,...,SGP,None,Mid,Female,Taiwan Athletics Open,7 - 8 June,2025,International,2025-06-11 16:38:50.923703,8 June/25
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31154,St. Andrew's Junior College,03:43.06,SAJC,None,5.0,A,4x400m Relay,None,None,None,...,None,None,Relay,Male,National School Games,28-04,2025,Local,2025-06-06 15:35:30.807106,28-04-25
31155,Catholic Junior College,03:43.67,CJC,None,6.0,A,4x400m Relay,None,None,None,...,None,None,Relay,Male,National School Games,28-04,2025,Local,2025-06-06 15:35:30.807106,28-04-25
31156,Yishun Innova Junior College,03:45.03,YIJC,None,7.0,A,4x400m Relay,None,None,None,...,None,None,Relay,Male,National School Games,28-04,2025,Local,2025-06-06 15:35:30.807106,28-04-25
31157,National Junior College,03:47.40,NJC,None,8.0,A,4x400m Relay,None,None,None,...,None,None,Relay,Male,National School Games,28-04,2025,Local,2025-06-06 15:35:30.807106,28-04-25


In [4073]:
# Calculate number of days from today to event date

competitors['event_date_dt'] = pd.to_datetime(competitors['event_date'], format='mixed', dayfirst=False)

competitors['delta_time']= datetime.datetime.now() - competitors['event_date_dt']

competitors['delta_time_conv'] = pd.to_numeric(competitors['delta_time'].dt.days, downcast='integer')

competitors['event_month'] = competitors['event_date_dt'].dt.month


In [4074]:
competitors

,NAME,RESULT,TEAM,AGE,COMPETITION_RANK,DIVISION,EVENT,DISTANCE,EVENT_CLASS,UNIQUE_ID,...,COMPETITION,DATE,YEAR,REGION,TIMESTAMP,event_date,event_date_dt,delta_time,delta_time_conv,event_month
0,Zubin Percy Muncherji,01:54.0,None,None,2.0,None,800m,None,None,None,...,Taiwan Athletics Open,7 - 8 June,2025,International,2025-06-11 16:38:50.923703,8 June/25,2025-06-08,11 days 22:20:39.219731,11.0,6.0
1,Nicole Low Sui Xuan,18:34.34,None,None,6.0,None,5000m,None,None,None,...,Taiwan Athletics Open,7 - 8 June,2025,International,2025-06-11 16:38:50.923703,8 June/25,2025-06-08,11 days 22:20:39.219731,11.0,6.0
2,Gabriel Lee,15.15,None,None,7.0,None,Triple Jump,None,None,None,...,Taiwan Athletics Open,7 - 8 June,2025,International,2025-06-11 16:38:50.923703,8 June/25,2025-06-08,11 days 22:20:39.219731,11.0,6.0
3,Esther Tay Shee Wei,3.2,None,None,10.0,None,Pole Vault,None,None,None,...,Taiwan Athletics Open,7 - 8 June,2025,International,2025-06-11 16:38:50.923703,8 June/25,2025-06-08,11 days 22:20:39.219731,11.0,6.0
4,Romaine Soh,02:16.90,None,None,5.0,None,800m,None,None,None,...,Taiwan Athletics Open,7 - 8 June,2025,International,2025-06-11 16:38:50.923703,8 June/25,2025-06-08,11 days 22:20:39.219731,11.0,6.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31154,St. Andrew's Junior College,03:43.06,SAJC,None,5.0,A,4x400m Relay,None,None,None,...,National School Games,28-04,2025,Local,2025-06-06 15:35:30.807106,28-04-25,2025-04-28,52 days 22:20:39.219731,52.0,4.0
31155,Catholic Junior College,03:43.67,CJC,None,6.0,A,4x400m Relay,None,None,None,...,National School Games,28-04,2025,Local,2025-06-06 15:35:30.807106,28-04-25,2025-04-28,52 days 22:20:39.219731,52.0,4.0
31156,Yishun Innova Junior College,03:45.03,YIJC,None,7.0,A,4x400m Relay,None,None,None,...,National School Games,28-04,2025,Local,2025-06-06 15:35:30.807106,28-04-25,2025-04-28,52 days 22:20:39.219731,52.0,4.0
31157,National Junior College,03:47.40,NJC,None,8.0,A,4x400m Relay,None,None,None,...,National School Games,28-04,2025,Local,2025-06-06 15:35:30.807106,28-04-25,2025-04-28,52 days 22:20:39.219731,52.0,4.0


In [4075]:
# Choose date range for SEAG qualification window from Oct 22 to current


#competitors = competitors[(competitors['delta_time_conv']>=0) & (competitors['delta_time_conv']<=365)]

#competitors=competitors.reset_index(drop=True)

start = datetime.datetime(2024, 6, 23)
#start = datetime.datetime(2025, 5, 1)


end = datetime.datetime(2025, 6, 10)

start_date = np.datetime64(start)
end_date = np.datetime64(end)


mask = (competitors['event_date_dt'] >= start_date) & (competitors['event_date_dt'] <= end_date)
athletes_selected = competitors.loc[mask]



In [4076]:
athletes_selected.to_csv('athletes_downloaded_AY.csv', encoding='utf-8')

In [4077]:
# Select all of 2024/25 for OCTC

#athletes_selected = competitors[(competitors['YEAR']=='2024')|(competitors['YEAR']=='2025')]

#athletes_selected = competitors[(competitors['YEAR']=='2025')]

In [4078]:
athletes_selected

,NAME,RESULT,TEAM,AGE,COMPETITION_RANK,DIVISION,EVENT,DISTANCE,EVENT_CLASS,UNIQUE_ID,...,COMPETITION,DATE,YEAR,REGION,TIMESTAMP,event_date,event_date_dt,delta_time,delta_time_conv,event_month
0,Zubin Percy Muncherji,01:54.0,None,None,2.0,None,800m,None,None,None,...,Taiwan Athletics Open,7 - 8 June,2025,International,2025-06-11 16:38:50.923703,8 June/25,2025-06-08,11 days 22:20:39.219731,11.0,6.0
1,Nicole Low Sui Xuan,18:34.34,None,None,6.0,None,5000m,None,None,None,...,Taiwan Athletics Open,7 - 8 June,2025,International,2025-06-11 16:38:50.923703,8 June/25,2025-06-08,11 days 22:20:39.219731,11.0,6.0
2,Gabriel Lee,15.15,None,None,7.0,None,Triple Jump,None,None,None,...,Taiwan Athletics Open,7 - 8 June,2025,International,2025-06-11 16:38:50.923703,8 June/25,2025-06-08,11 days 22:20:39.219731,11.0,6.0
3,Esther Tay Shee Wei,3.2,None,None,10.0,None,Pole Vault,None,None,None,...,Taiwan Athletics Open,7 - 8 June,2025,International,2025-06-11 16:38:50.923703,8 June/25,2025-06-08,11 days 22:20:39.219731,11.0,6.0
4,Romaine Soh,02:16.90,None,None,5.0,None,800m,None,None,None,...,Taiwan Athletics Open,7 - 8 June,2025,International,2025-06-11 16:38:50.923703,8 June/25,2025-06-08,11 days 22:20:39.219731,11.0,6.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31154,St. Andrew's Junior College,03:43.06,SAJC,None,5.0,A,4x400m Relay,None,None,None,...,National School Games,28-04,2025,Local,2025-06-06 15:35:30.807106,28-04-25,2025-04-28,52 days 22:20:39.219731,52.0,4.0
31155,Catholic Junior College,03:43.67,CJC,None,6.0,A,4x400m Relay,None,None,None,...,National School Games,28-04,2025,Local,2025-06-06 15:35:30.807106,28-04-25,2025-04-28,52 days 22:20:39.219731,52.0,4.0
31156,Yishun Innova Junior College,03:45.03,YIJC,None,7.0,A,4x400m Relay,None,None,None,...,National School Games,28-04,2025,Local,2025-06-06 15:35:30.807106,28-04-25,2025-04-28,52 days 22:20:39.219731,52.0,4.0
31157,National Junior College,03:47.40,NJC,None,8.0,A,4x400m Relay,None,None,None,...,National School Games,28-04,2025,Local,2025-06-06 15:35:30.807106,28-04-25,2025-04-28,52 days 22:20:39.219731,52.0,4.0


In [4079]:
athletes_selected[athletes_selected['NAME']=='Hia, Caleb']

,NAME,RESULT,TEAM,AGE,COMPETITION_RANK,DIVISION,EVENT,DISTANCE,EVENT_CLASS,UNIQUE_ID,...,COMPETITION,DATE,YEAR,REGION,TIMESTAMP,event_date,event_date_dt,delta_time,delta_time_conv,event_month


In [4080]:
# Choose 2024/25 only

athletes = athletes_selected

In [4081]:
athletes

,NAME,RESULT,TEAM,AGE,COMPETITION_RANK,DIVISION,EVENT,DISTANCE,EVENT_CLASS,UNIQUE_ID,...,COMPETITION,DATE,YEAR,REGION,TIMESTAMP,event_date,event_date_dt,delta_time,delta_time_conv,event_month
0,Zubin Percy Muncherji,01:54.0,None,None,2.0,None,800m,None,None,None,...,Taiwan Athletics Open,7 - 8 June,2025,International,2025-06-11 16:38:50.923703,8 June/25,2025-06-08,11 days 22:20:39.219731,11.0,6.0
1,Nicole Low Sui Xuan,18:34.34,None,None,6.0,None,5000m,None,None,None,...,Taiwan Athletics Open,7 - 8 June,2025,International,2025-06-11 16:38:50.923703,8 June/25,2025-06-08,11 days 22:20:39.219731,11.0,6.0
2,Gabriel Lee,15.15,None,None,7.0,None,Triple Jump,None,None,None,...,Taiwan Athletics Open,7 - 8 June,2025,International,2025-06-11 16:38:50.923703,8 June/25,2025-06-08,11 days 22:20:39.219731,11.0,6.0
3,Esther Tay Shee Wei,3.2,None,None,10.0,None,Pole Vault,None,None,None,...,Taiwan Athletics Open,7 - 8 June,2025,International,2025-06-11 16:38:50.923703,8 June/25,2025-06-08,11 days 22:20:39.219731,11.0,6.0
4,Romaine Soh,02:16.90,None,None,5.0,None,800m,None,None,None,...,Taiwan Athletics Open,7 - 8 June,2025,International,2025-06-11 16:38:50.923703,8 June/25,2025-06-08,11 days 22:20:39.219731,11.0,6.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31154,St. Andrew's Junior College,03:43.06,SAJC,None,5.0,A,4x400m Relay,None,None,None,...,National School Games,28-04,2025,Local,2025-06-06 15:35:30.807106,28-04-25,2025-04-28,52 days 22:20:39.219731,52.0,4.0
31155,Catholic Junior College,03:43.67,CJC,None,6.0,A,4x400m Relay,None,None,None,...,National School Games,28-04,2025,Local,2025-06-06 15:35:30.807106,28-04-25,2025-04-28,52 days 22:20:39.219731,52.0,4.0
31156,Yishun Innova Junior College,03:45.03,YIJC,None,7.0,A,4x400m Relay,None,None,None,...,National School Games,28-04,2025,Local,2025-06-06 15:35:30.807106,28-04-25,2025-04-28,52 days 22:20:39.219731,52.0,4.0
31157,National Junior College,03:47.40,NJC,None,8.0,A,4x400m Relay,None,None,None,...,National School Games,28-04,2025,Local,2025-06-06 15:35:30.807106,28-04-25,2025-04-28,52 days 22:20:39.219731,52.0,4.0


In [4082]:
athletes[athletes['NAME']=='Jun Jie Calvin Quek']

,NAME,RESULT,TEAM,AGE,COMPETITION_RANK,DIVISION,EVENT,DISTANCE,EVENT_CLASS,UNIQUE_ID,...,COMPETITION,DATE,YEAR,REGION,TIMESTAMP,event_date,event_date_dt,delta_time,delta_time_conv,event_month
18,Jun Jie Calvin Quek,50.94,None,None,8,None,400m Hurdles,None,None,None,...,26th Asian Athletics Championships,31-May,2025,International,36:03.8,31-May-25,2025-05-31,19 days 22:20:39.219731,19.0,5.0
31,Jun Jie Calvin Quek,50.58,None,None,2,None,400m Hurdles,None,None,None,...,26th Asian Athletics Championships,27 - 31 May,2025,International,2025-06-01 16:49:06.835826,31 May 25,2025-05-31,19 days 22:20:39.219731,19.0,5.0
4333,Jun Jie Calvin Quek,51.64,None,None,6.0,None,400m Hurdles,None,None,None,...,12th Kinami Michitaka Memorial Athletics Meet,11-May,2025,International,2025-05-16 15:18:11.207945,11-May-25,2025-05-11,39 days 22:20:39.219731,39.0,5.0
5424,Jun Jie Calvin Quek,51.79,None,None,2,None,400m Hurdles,None,None,None,...,XXXIII Qosanov Memorial,22 - 23 June,2024,International,None,23 June 24,2024-06-23,361 days 22:20:39.219731,361.0,6.0
7485,Jun Jie Calvin Quek,10.86,None,None,2,None,100m,None,None,None,...,Potch Invitational Meet,16-Apr,2025,International,None,16-Apr-25,2025-04-16,64 days 22:20:39.219731,64.0,4.0
19361,Jun Jie Calvin Quek,50.77,None,None,1,None,400m Hurdles,None,None,None,...,Potch Invitational Meet,16-Apr,2025,International,None,16-Apr-25,2025-04-16,64 days 22:20:39.219731,64.0,4.0
22617,Jun Jie Calvin Quek,52.62,None,None,9,None,400m Hurdles,None,None,None,...,Sydney Track Classic,15-Mar,2025,International,None,15-Mar-25,2025-03-15,96 days 22:20:39.219731,96.0,3.0
23631,Jun Jie Calvin Quek,56.95,None,None,3,None,400m Hurdles,None,None,None,...,XXXIII Qosanov Memorial,22 - 23 June,2024,International,None,23 June 24,2024-06-23,361 days 22:20:39.219731,361.0,6.0


In [4083]:
# Process dates to extract age

# Map NSG divisions into age

mask = (athletes['DIVISION'].str.contains(r'A', na=False))
athletes.loc[mask, 'AGE'] = '18.5'

mask = (athletes['DIVISION'].str.contains(r'B', na=False))
athletes.loc[mask, 'AGE'] = '16'

mask = (athletes['DIVISION'].str.contains(r'C', na=False))
athletes.loc[mask, 'AGE'] = '13.5'

mask = (athletes['DIVISION'].str.contains(r'O', na=False))
athletes.loc[mask, 'AGE'] = '12'


In [4084]:
def length(string):

    B = ''
    year = ''

    try:

        length = len(string)

        if length == 2:

            string = '19' + string

        elif length == 1:

            string = ''

        else:

            pass

        if string is not None or len(string) != 1:

            B = parser.parse(string, dayfirst=True)
                        
    except:

        pass

    return B


athletes['DOB_dt'] = athletes['DOB'].apply(length)




/var/folders/q5/yf8g5p896_b94gkbhqcjx3t40000gn/T/ipykernel_51640/1704523743.py:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  athletes['DOB_dt'] = athletes['DOB'].apply(length)


In [4088]:
os.chdir('/Users/veesheenyuen/Desktop/DataScience/SAA/Asian Youth/')


athletes.to_csv('age_post_map_AY.csv', sep=',', encoding='utf-8-sig', index=False)


In [4089]:
# Create temporary mapped event column
# Junior Category Only

athletes['MAPPED_EVENT']=''

for col in athletes.columns:
    athletes[col] = athletes[col].astype(str)
    athletes[col] = athletes[col].str.replace('\xa0', ' ', regex=True)
    athletes[col] = athletes[col].str.replace('[\x00-\x1f\x7f-\x9f]', '', regex=True)
    athletes[col] = athletes[col].str.replace('\r', ' ', regex=True)
    athletes[col] = athletes[col].str.replace('\n', ' ', regex=True)
    athletes[col] = athletes[col].str.strip()


# Correct javelin category

mask = athletes['EVENT'].str.contains(r'Javelin', na=True)
athletes.loc[mask, 'CATEGORY_EVENT'] = 'Throw'


# Running

mask = (athletes['EVENT'].str.contains(r'Dash', na=True) & athletes['DISTANCE'].str.contains(r'100', na=True))
athletes.loc[mask, 'MAPPED_EVENT'] = '100m'
mask = (athletes['EVENT'].str.contains(r'Run', na=True) & athletes['DISTANCE'].str.contains(r'100', na=True))
athletes.loc[mask, 'MAPPED_EVENT'] = '100m'
mask = athletes['EVENT'].str.contains(r'100 Meter Run', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = '100m'
mask = athletes['EVENT'].str.contains(r'^100m$', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = '100m'

mask = (athletes['EVENT'].str.contains(r'Dash', na=True) & athletes['DISTANCE'].str.contains(r'200', na=True))
athletes.loc[mask, 'MAPPED_EVENT'] = '200m'
mask = (athletes['EVENT'].str.contains(r'Run', na=True) & athletes['DISTANCE'].str.contains(r'200', na=True))
athletes.loc[mask, 'MAPPED_EVENT'] = '200m'
mask = athletes['EVENT'].str.contains(r'^200m$', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = '200m'
mask = athletes['EVENT'].str.contains(r'200\sMeter', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = '200m'

mask = (athletes['EVENT'].str.contains(r'Dash', na=True) & athletes['DISTANCE'].str.contains(r'400', na=True))
athletes.loc[mask, 'MAPPED_EVENT'] = '400m'
mask = athletes['EVENT'].str.contains(r'^400m$', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = '400m'
mask = athletes['EVENT'].str.contains(r'^400\sMeter$', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = '400m'
mask = (athletes['EVENT'].str.contains(r'Run', na=True) & athletes['DISTANCE'].str.contains(r'400', na=True))
athletes.loc[mask, 'MAPPED_EVENT'] = '400m'


mask = (athletes['EVENT'].str.contains(r'Run', na=True) & athletes['DISTANCE'].str.contains(r'800', na=True))
athletes.loc[mask, 'MAPPED_EVENT'] = '800m'
mask = athletes['EVENT'].str.contains(r'800 Meter Run', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = '800m'
mask = athletes['EVENT'].str.contains(r'^800m$', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = '800m'
mask = (athletes['EVENT'].str.contains(r'Run', na=True) & athletes['DISTANCE'].str.contains(r'1000', na=True))
athletes.loc[mask, 'MAPPED_EVENT'] = '1000m'


mask = (athletes['EVENT'].str.contains(r'Run', na=True) & athletes['DISTANCE'].str.contains(r'1500', na=True))
athletes.loc[mask, 'MAPPED_EVENT'] = '1500m'
mask = athletes['EVENT'].str.contains(r'^1500m$', na=True, regex=True)
athletes.loc[mask, 'MAPPED_EVENT'] = '1500m'
mask = (athletes['EVENT'].str.contains(r'Run', na=True) & athletes['DISTANCE'].str.contains(r'3000', na=True))
athletes.loc[mask, 'MAPPED_EVENT'] = '3000m'
#mask = athletes['EVENT'].str.contains(r'3000m', na=True)
#athletes.loc[mask, 'MAPPED_EVENT'] = '3000m'
mask = (athletes['EVENT'].str.contains(r'Run', na=True) & athletes['DISTANCE'].str.contains(r'5000', na=True))
athletes.loc[mask, 'MAPPED_EVENT'] = '5000m'
mask = athletes['EVENT'].str.contains(r'^5000m$', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = '5000m'
mask = (athletes['EVENT'].str.contains(r'Run', na=True) & athletes['DISTANCE'].str.contains(r'10000', na=True))
athletes.loc[mask, 'MAPPED_EVENT'] = '10,000m'
mask = athletes['EVENT'].str.contains(r'^10000m$', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = '10,000m'
mask = athletes['EVENT'].str.contains(r'^10\,000m$', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = '10,000m'
mask = (athletes['EVENT'].str.contains(r'Run', na=True) & athletes['DISTANCE'].str.contains(r'Mile', na=True))
athletes.loc[mask, 'MAPPED_EVENT'] = '1 Mile'

#mask = athletes['EVENT'].str.contains(r'10\,000m', na=True, regex=True)
#athletes.loc[mask, 'MAPPED_EVENT'] = '10000m'



# Hurdles

#mask = athletes['EVENT'].str.contains(r'100\sMeter\sHurdles\s\(0\.838m\)', na=True)
#athletes.loc[mask, 'MAPPED_EVENT'] = '100m Hurdles'
#mask = athletes['EVENT'].str.contains(r'100m\sHurdles\s\(0\.838m\)', na=True)
#athletes.loc[mask, 'MAPPED_EVENT'] = '100m Hurdles'


mask = athletes['EVENT'].str.contains(r'110m\sHurdles\s\(0\.914m\)', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = '110m hurdles'
##mask = athletes['EVENT'].str.contains(r'110m\sHurdles\s\(1\.067m\)', na=True)
##athletes.loc[mask, 'MAPPED_EVENT'] = '110m Hurdles'


mask = (athletes['EVENT'].str.contains(r'100m Hurdles|100m hurdles', na=False) & athletes['EVENT_CLASS'].str.contains('0.762', na=False) & athletes['GENDER'].str.contains(r'Female', na=False))  # this is the correct syntax
athletes.loc[mask, 'MAPPED_EVENT'] = '100m Hurdles'
#mask = (athletes['EVENT'].str.contains(r'100m Hurdles|100m hurdles', na=False) & athletes['DIVISION'].str.contains('None', na=False) & athletes['GENDER'].str.contains(r'Female', na=False) & athletes['REGION'].str.contains(r'International', na=False))  # this is the correct syntax
#athletes.loc[mask, 'MAPPED_EVENT'] = '100m Hurdles'
mask = (athletes['EVENT'].str.contains(r'^Hurdles$', na=False) & athletes['DISTANCE'].str.contains(r'100', na=False) & athletes['EVENT_CLASS'].str.contains(r'0.762', na=False) & athletes['GENDER'].str.contains(r'Female', na=False))
athletes.loc[mask, 'MAPPED_EVENT'] = '100m Hurdles'
#mask = (athletes['EVENT'].str.contains(r'100m Hurdles|100m hurdles', na=False) & athletes['REGION'].str.contains(r'International', na=False) & athletes['GENDER'].str.contains(r'Female', na=False))
#athletes.loc[mask, 'MAPPED_EVENT'] = '100m Hurdles'




# Using np.where instead

#athletes['MAPPED_EVENT'] = np.where(((athletes['EVENT']=='110m hurdles|110m Hurdles') & ((athletes['EVENT_CLASS']=='')|athletes['EVENT_CLASS']=='None') & (athletes['REGION']=='International')), '110m Hurdles', ' ')   
                                
mask = (athletes['EVENT'].str.contains(r'110m Hurdles|110m hurdles', na=False) & athletes['EVENT_CLASS'].str.contains(r'0.914|91.4', na=False) & athletes['GENDER'].str.contains(r'Male', na=False))
athletes.loc[mask, 'MAPPED_EVENT'] = '110m Hurdles'


mask = (athletes['EVENT'].str.contains(r'^Hurdles$', na=False) & athletes['DISTANCE'].str.contains(r'110', na=False) & athletes['EVENT_CLASS'].str.contains(r'0.914', na=False))
athletes.loc[mask, 'MAPPED_EVENT'] = '110m Hurdles'
mask = (athletes['EVENT'].str.contains(r'^Hurdles$', na=False) & athletes['DISTANCE'].str.contains(r'110', na=False) & athletes['EVENT_CLASS'].str.contains(r'0.762', na=False) & athletes['GENDER'].str.contains(r'Female', na=False))
athletes.loc[mask, 'MAPPED_EVENT'] = '110m Hurdles'
mask = (athletes['EVENT'].str.contains(r'^Hurdles$', na=False) & athletes['DISTANCE'].str.contains(r'110', na=False) & athletes['DIVISION'].str.contains(r'U18', na=False))
athletes.loc[mask, 'MAPPED_EVENT'] = '110m Hurdles'

#mask = (athletes['EVENT'].str.contains(r'^Hurdles$', na=False) & athletes['DISTANCE'].str.contains(r'110', na=False) & athletes['EVENT_CLASS'].str.contains(' ', na=True))
#athletes.loc[mask, 'MAPPED_EVENT'] = '110m Hurdles'







#mask = (athletes['EVENT'].str.contains(r'^400m\sHurdles$', na=False) & athletes['EVENT_CLASS'].str.contains(r'', na=False))
#athletes.loc[mask, 'MAPPED_EVENT'] = '400m Hurdles'
#mask = athletes['EVENT'].str.contains(r'400m\sHurdles\s\(0.840m\)', na=True)
#athletes.loc[mask, 'MAPPED_EVENT'] = ' '
mask = (athletes['EVENT'].str.contains(r'^Hurdles$', na=False) & athletes['DISTANCE'].str.contains(r'400', na=False) & athletes['EVENT_CLASS'].str.contains(r'0.762|76.2cm', na=False) & athletes['GENDER'].str.contains(r'Female', na=False))
athletes.loc[mask, 'MAPPED_EVENT'] = '400m Hurdles'


mask = (athletes['EVENT'].str.contains(r'^Hurdles$', na=False) & athletes['DISTANCE'].str.contains(r'400', na=False) & athletes['EVENT_CLASS'].str.contains(r'0.84|84|0.838', na=False))
athletes.loc[mask, 'MAPPED_EVENT'] = '400m Hurdles'
mask = (athletes['EVENT'].str.contains(r'^Hurdles$', na=False) & athletes['DISTANCE'].str.contains(r'400', na=False) & athletes['DIVISION'].str.contains(r'U18', na=False))
athletes.loc[mask, 'MAPPED_EVENT'] = '400m Hurdles'

mask = (athletes['EVENT'].str.contains(r'400m Hurdles', na=False) & athletes['EVENT_CLASS'].str.contains(r'0.84|84|0.838', na=False)  & athletes['GENDER'].str.contains(r'Male', na=False))
athletes.loc[mask, 'MAPPED_EVENT'] = '400m Hurdles'


#mask = athletes['EVENT'].str.contains(r'^400\sMeter\sHurdles\s\(0\.762m\)$', na=True)
#athletes.loc[mask, 'MAPPED_EVENT'] = '400m Hurdles'
##mask = athletes['EVENT'].str.contains(r'^400m\sHurdles\s\(0\.762m\)$', na=True)
#athletes.loc[mask, 'MAPPED_EVENT'] = '400m Hurdles'
mask = (athletes['EVENT'].str.contains(r'Hurdles', na=False) & athletes['DISTANCE'].str.contains(r'400', na=False) & athletes['EVENT_CLASS'].str.contains(r'0.762', na=False)& athletes['GENDER'].str.contains(r'Female', na=False))
athletes.loc[mask, 'MAPPED_EVENT'] = '400m Hurdles'
mask = (athletes['EVENT'].str.contains(r'400m Hurdles', na=False) & athletes['EVENT_CLASS'].str.contains(r'0.762m', na=False) & athletes['GENDER'].str.contains(r'Female', na=False))
athletes.loc[mask, 'MAPPED_EVENT'] = '400m Hurdles'
mask = (athletes['EVENT'].str.contains(r'400m Hurdles|400m hurdles', na=False) & athletes['EVENT_CLASS'].str.contains('0.762|0.84|0.838', na=False) & athletes['REGION'].str.contains(r'International', na=False))  # this is the correct syntax
athletes.loc[mask, 'MAPPED_EVENT'] = '400m Hurdles'
#mask = (athletes['EVENT'].str.contains(r'400m Hurdles|400m hurdles', na=False) & athletes['REGION'].str.contains(r'International', na=False))  # this is the correct syntax
#athletes.loc[mask, 'MAPPED_EVENT'] = '400m Hurdles'



# Throws


mask = (athletes['EVENT'].str.contains(r'Javelin Throw|Javelin throw|Javelin', na=False) & athletes['EVENT_CLASS'].str.contains(r'500g', na=False) & athletes['GENDER'].str.contains(r'Female', na=False))
athletes.loc[mask, 'MAPPED_EVENT'] = 'Javelin Throw'
mask = (athletes['EVENT'].str.contains(r'Javelin Throw|Javelin throw|Javelin', na=False) & athletes['EVENT_CLASS'].str.contains(r'700g', na=False) & athletes['GENDER'].str.contains(r'Male', na=False))
athletes.loc[mask, 'MAPPED_EVENT'] = 'Javelin Throw'
#mask = (athletes['EVENT'].str.contains(r'Javelin Throw|Javelin throw', na=False) & athletes['DIVISION'].str.contains(r'OPEN|Open', na=False))
#athletes.loc[mask, 'MAPPED_EVENT'] = 'Javelin Throw'

mask = (athletes['EVENT'].str.contains(r'Shot Put|Shot put', na=False, regex=True) & (athletes['GENDER']=='Female') & (athletes['EVENT_CLASS']=='3kg'))# there are some additional characters after Put
athletes.loc[mask, 'MAPPED_EVENT'] = 'Shot Put'


#mask = athletes['EVENT'].str.contains(r'Women\sShot\sPut\s4kg\sOpen', na=True, regex=True)
#athletes.loc[mask, 'MAPPED_EVENT'] = 'Shot Put'
#mask = athletes['EVENT'].str.contains(r'Men\sShot\sPut\s4kg\sOPEN', na=True, regex=True)
#athletes.loc[mask, 'MAPPED_EVENT'] = 'Shot put'


#mask = athletes['EVENT'].str.contains(r'Women\sShot\sPut\s\(4kg\)', na=True, regex=True)
#athletes.loc[mask, 'MAPPED_EVENT'] = 'Shot Put'
#mask = athletes['EVENT'].str.contains(r'Shot\sPut\s\(7\.26kg\)', na=True, regex=True)
#athletes.loc[mask, 'MAPPED_EVENT'] = 'Shot Put'
#mask = athletes['EVENT'].str.contains(r'Shot\sPut\s7\.26kg\sOpen', na=True, regex=True)
#athletes.loc[mask, 'MAPPED_EVENT'] = 'Shot Put'
mask = (athletes['EVENT'].str.contains(r'Shot Put|Shot put', na=False) & (athletes['GENDER']=='Male') & (athletes['EVENT_CLASS'].str.contains(r'5', na=False)))# there are some additional characters after Put
athletes.loc[mask, 'MAPPED_EVENT'] = 'Shot Put'
mask = (athletes['EVENT'].str.contains(r'Shot Put|Shot put', na=False) & (athletes['GENDER']=='Female') & (athletes['EVENT_CLASS'].str.contains(r'3', na=False)))# there are some additional characters after Put
athletes.loc[mask, 'MAPPED_EVENT'] = 'Shot Put'

#mask = (athletes['EVENT'].str.contains(r'Shot Put|Shot put', na=False) & (athletes['DIVISION'].str.contains(r'OPEN|Open', na=False)))# there are some additional characters after Put
#athletes.loc[mask, 'MAPPED_EVENT'] = 'Shot Put'

#mask = (athletes['EVENT'].str.contains(r'Shot Put|Shot put', na=False) & (athletes['REGION'].str.contains(r'International', na=False)) & athletes['EVENT_CLASS'].str.contains(r'None', na=False))# there are some additional characters after Put
#athletes.loc[mask, 'MAPPED_EVENT'] = 'Shot Put'



#mask = (athletes['EVENT'].str.contains(r'Hammer\sThrow\s\(4kg\)', na=True) & (athletes['GENDER']=='Female'))
#athletes.loc[mask, 'MAPPED_EVENT'] = 'Hammer Throw'
#mask = athletes['EVENT'].str.contains(r'Hammer\sThrow\s\(7\.26kg\)', na=True)
#athletes.loc[mask, 'MAPPED_EVENT'] = 'Hammer Throw'
mask = (athletes['EVENT'].str.contains(r'Hammer Throw|Hammer throw', na=False) & athletes['EVENT_CLASS'].str.contains(r'5kg', na=False))
athletes.loc[mask, 'MAPPED_EVENT'] = 'Hammer Throw'
mask = (athletes['EVENT'].str.contains(r'Hammer Throw|Hammer throw', na=False) & athletes['EVENT_CLASS'].str.contains(r'3.00kg', na=False))
athletes.loc[mask, 'MAPPED_EVENT'] = 'Hammer Throw'
#mask = (athletes['EVENT'].str.contains(r'Hammer Throw|Hammer throw', na=False) & (athletes['DIVISION'].str.contains(r'OPEN|Open', na=False)))# there are some additional characters after Put
#athletes.loc[mask, 'MAPPED_EVENT'] = 'Hammer Throw'



mask = (athletes['EVENT'].str.contains(r'Discus Throw|Discus|Discus throw', na=False) & athletes['EVENT_CLASS'].str.contains(r'1.5kg|1.50kg', na=False) & athletes['GENDER'].str.contains(r'Male', na=False))
athletes.loc[mask, 'MAPPED_EVENT'] = 'Discus Throw'
mask = (athletes['EVENT'].str.contains(r'Discus Throw|Discus|Discus throw', na=False) & athletes['EVENT_CLASS'].str.contains(r'1kg|1.00kg', na=False) & athletes['GENDER'].str.contains(r'Female', na=False))
athletes.loc[mask, 'MAPPED_EVENT'] = 'Discus Throw'

#mask = (athletes['EVENT'].str.contains(r'Discus Throw|Discus throw', na=False) & athletes['REGION'].str.contains(r'International', na=False))
#athletes.loc[mask, 'MAPPED_EVENT'] = 'Discus Throw'
mask = (athletes['EVENT'].str.contains(r'Discus Throw|Discus throw', na=False) & athletes['DIVISION'].str.contains(r'U18', na=False))
athletes.loc[mask, 'MAPPED_EVENT'] = 'Discus Throw'
#mask = (athletes['EVENT'].str.contains(r'Discus Throw|Discus throw', na=False) & athletes['DIVISION'].str.contains(r'None', na=False) & athletes['EVENT_CLASS'].str.contains(r'None', na=False))
#athletes.loc[mask, 'MAPPED_EVENT'] = 'Discus Throw'


# Jumps

mask = athletes['EVENT'].str.contains(r'High Jump', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = 'High Jump'

mask = athletes['EVENT'].str.contains(r'^Long\sJump$', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = 'Long Jump'
mask = athletes['EVENT'].str.contains(r'Long Jump Open', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = 'Long Jump'
mask = athletes['EVENT'].str.contains(r'Long Jump Trial', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = 'Long Jump'


mask = athletes['EVENT'].str.contains(r'Triple Jump', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = 'Triple Jump'
mask = athletes['EVENT'].str.contains(r'Pole Vault', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = 'Pole Vault'
mask = athletes['EVENT'].str.contains(r'High jump', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = 'High Jump'
mask = athletes['EVENT'].str.contains(r'Long jump', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = 'Long Jump'
mask = athletes['EVENT'].str.contains(r'Triple jump', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = 'Triple Jump'
mask = athletes['EVENT'].str.contains(r'^Pole\svault$', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = 'Pole Vault'

# Steeplechase

#mask = athletes['EVENT'].str.contains(r'2000m S/C', na=True)
#athletes.loc[mask, 'MAPPED_EVENT'] = '2000m Steeplechase'
#mask = athletes['EVENT'].str.contains(r'2000m steeplechase', na=True)
#athletes.loc[mask, 'MAPPED_EVENT'] = '2000m Steeplechase'
#mask = athletes['EVENT'].str.contains(r'2000 Meter Steeplechase', na=True)
#athletes.loc[mask, 'MAPPED_EVENT'] = '2000m Steeplechase'
#mask = (athletes['EVENT'].str.contains(r'2000m Steeplechase|2000m S\/C', na=True) & athletes['REGION'].str.contains(r'International', na=False))
#athletes.loc[mask, 'MAPPED_EVENT'] = '2000m Steeplechase'
mask = (athletes['EVENT'].str.contains(r'Steeplechase|steeplechase|S\/C', na=False) & athletes['DISTANCE'].str.contains(r'2000', na=False)  & athletes['EVENT_CLASS'].str.contains(r'0.914', na=False)  & athletes['GENDER'].str.contains(r'Male', na=False))
athletes.loc[mask, 'MAPPED_EVENT'] = '2000m Steeplechase'
mask = (athletes['EVENT'].str.contains(r'Steeplechase|steeplechase|S\/C', na=False) & athletes['DISTANCE'].str.contains(r'2000', na=False)  & athletes['EVENT_CLASS'].str.contains(r'0.762', na=False)  & athletes['GENDER'].str.contains(r'Female', na=False))
athletes.loc[mask, 'MAPPED_EVENT'] = '2000m Steeplechase'


mask = (athletes['EVENT'].str.contains(r'2000m Steeplechase|2000m steeplechase|2000m S\/C', na=False) & athletes['EVENT_CLASS'].str.contains(r'0.914', na=False) & athletes['GENDER'].str.contains(r'Male', na=False))
athletes.loc[mask, 'MAPPED_EVENT'] = '2000m Steeplechase'
mask = (athletes['EVENT'].str.contains(r'2000m Steeplechase|2000m steeplechase|2000m S\/C', na=False) & athletes['EVENT_CLASS'].str.contains(r'0.762', na=False) & athletes['GENDER'].str.contains(r'Female', na=False))
athletes.loc[mask, 'MAPPED_EVENT'] = '2000m Steeplechase'


#mask = (athletes['EVENT'].str.contains(r'Steeplechase', na=False) & athletes['DISTANCE'].str.contains(r'2000', na=False)  & athletes['DIVISION'].str.contains(r'OPEN|Open', na=False))
#athletes.loc[mask, 'MAPPED_EVENT'] = '2000m Steeplechase'


# Marathon

mask = athletes['EVENT'].str.contains(r'^Marathon$', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = 'Marathon'
mask = athletes['EVENT'].str.contains(r'^Half\sMarathon$', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = 'Half Marathon'
mask = athletes['EVENT'].str.contains(r'^Half\smarathon$', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = 'Half Marathon'


# Walk

mask = (athletes['EVENT'].str.contains(r'Race Walk', na=False) & athletes['DISTANCE'].str.contains(r'5000', na=False))
athletes.loc[mask, 'MAPPED_EVENT'] = '5000m Racewalk'

mask = (athletes['EVENT'].str.contains(r'Race Walk', na=False) & athletes['DISTANCE'].str.contains(r'10000', na=False))
athletes.loc[mask, 'MAPPED_EVENT'] = '10000m Racewalk'


#mask = athletes['EVENT'].str.contains(r'10000 Meter Race Walk', na=True)
#athletes.loc[mask, 'MAPPED_EVENT'] = '10000m Race Walk'

# Relay

mask = athletes['EVENT'].str.contains(r'4x80m Relay', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = '4 x 80m'
mask = athletes['EVENT'].str.contains(r'^4\sx\s100m$', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = '4 x 100m'
mask = athletes['EVENT'].str.contains(r'4x100m Relay', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = '4 x 100m'
mask = athletes['EVENT'].str.contains(r'4 X 100m Relay', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = '4 x 100m'
mask = (athletes['EVENT'].str.contains(r'Relay', na=False) & athletes['DISTANCE'].str.contains(r'400', na=False))
athletes.loc[mask, 'MAPPED_EVENT'] = '4 x 100m'

mask = athletes['EVENT'].str.contains(r'4x400m Relay', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = '4 x 400m'
mask = athletes['EVENT'].str.contains(r'4 X 400m Relay', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = '4 x 400m'
mask = athletes['EVENT'].str.contains(r'4x100 Meter Relay', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = '4 x 100m'
mask = (athletes['EVENT'].str.contains(r'Relay', na=False) & athletes['DISTANCE'].str.contains(r'1600', na=False))
athletes.loc[mask, 'MAPPED_EVENT'] = '4 x 400m'
mask = athletes['EVENT'].str.contains(r'^4\sx\s400m$', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = '4 x 400m'

# Decathlon/Heptathlon

mask = athletes['EVENT'].str.contains(r'^Heptathlon$', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = 'Heptathlon'
mask = athletes['EVENT'].str.contains(r'^Decathlon$', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = 'Decathlon'
mask = athletes['EVENT'].str.contains(r'Heptathlon', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = 'Heptathlon'
mask = athletes['EVENT'].str.contains(r'Decathlon', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = 'Decathlon'



/var/folders/q5/yf8g5p896_b94gkbhqcjx3t40000gn/T/ipykernel_51640/3522591407.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  athletes['MAPPED_EVENT']=''
/var/folders/q5/yf8g5p896_b94gkbhqcjx3t40000gn/T/ipykernel_51640/3522591407.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  athletes[col] = athletes[col].astype(str)
/var/folders/q5/yf8g5p896_b94gkbhqcjx3t40000gn/T/ipykernel_51640/3522591407.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .l

/var/folders/q5/yf8g5p896_b94gkbhqcjx3t40000gn/T/ipykernel_51640/3522591407.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  athletes[col] = athletes[col].str.replace('\xa0', ' ', regex=True)
/var/folders/q5/yf8g5p896_b94gkbhqcjx3t40000gn/T/ipykernel_51640/3522591407.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  athletes[col] = athletes[col].str.replace('[\x00-\x1f\x7f-\x9f]', '', regex=True)
/var/folders/q5/yf8g5p896_b94gkbhqcjx3t40000gn/T/ipykernel_51640/3522591407.py:10: SettingWithCopyWarning: 

/var/folders/q5/yf8g5p896_b94gkbhqcjx3t40000gn/T/ipykernel_51640/3522591407.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  athletes[col] = athletes[col].str.replace('\n', ' ', regex=True)
/var/folders/q5/yf8g5p896_b94gkbhqcjx3t40000gn/T/ipykernel_51640/3522591407.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  athletes[col] = athletes[col].str.strip()
/var/folders/q5/yf8g5p896_b94gkbhqcjx3t40000gn/T/ipykernel_51640/3522591407.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of

/var/folders/q5/yf8g5p896_b94gkbhqcjx3t40000gn/T/ipykernel_51640/3522591407.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  athletes[col] = athletes[col].str.replace('\r', ' ', regex=True)
/var/folders/q5/yf8g5p896_b94gkbhqcjx3t40000gn/T/ipykernel_51640/3522591407.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  athletes[col] = athletes[col].str.replace('\n', ' ', regex=True)
/var/folders/q5/yf8g5p896_b94gkbhqcjx3t40000gn/T/ipykernel_51640/3522591407.py:12: SettingWithCopyWarning: 
A value is tryin

In [4090]:
for col in athletes.columns:
    athletes[col] = athletes[col].astype(str)
    athletes[col] = athletes[col].str.replace('\xa0', ' ', regex=True)
    athletes[col] = athletes[col].str.replace('[\x00-\x1f\x7f-\x9f]', '', regex=True)
    athletes[col] = athletes[col].str.replace('\r', ' ', regex=True)
    athletes[col] = athletes[col].str.replace('\n', ' ', regex=True)
    athletes[col] = athletes[col].str.strip()

/var/folders/q5/yf8g5p896_b94gkbhqcjx3t40000gn/T/ipykernel_51640/2203170124.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  athletes[col] = athletes[col].astype(str)
/var/folders/q5/yf8g5p896_b94gkbhqcjx3t40000gn/T/ipykernel_51640/2203170124.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  athletes[col] = athletes[col].str.replace('\xa0', ' ', regex=True)
/var/folders/q5/yf8g5p896_b94gkbhqcjx3t40000gn/T/ipykernel_51640/2203170124.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of

/var/folders/q5/yf8g5p896_b94gkbhqcjx3t40000gn/T/ipykernel_51640/2203170124.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  athletes[col] = athletes[col].str.replace('\n', ' ', regex=True)
/var/folders/q5/yf8g5p896_b94gkbhqcjx3t40000gn/T/ipykernel_51640/2203170124.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  athletes[col] = athletes[col].str.strip()
/var/folders/q5/yf8g5p896_b94gkbhqcjx3t40000gn/T/ipykernel_51640/2203170124.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a

/var/folders/q5/yf8g5p896_b94gkbhqcjx3t40000gn/T/ipykernel_51640/2203170124.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  athletes[col] = athletes[col].str.replace('\xa0', ' ', regex=True)
/var/folders/q5/yf8g5p896_b94gkbhqcjx3t40000gn/T/ipykernel_51640/2203170124.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  athletes[col] = athletes[col].str.replace('[\x00-\x1f\x7f-\x9f]', '', regex=True)
/var/folders/q5/yf8g5p896_b94gkbhqcjx3t40000gn/T/ipykernel_51640/2203170124.py:5: SettingWithCopyWarning: 


In [4091]:
os.chdir('/Users/veesheenyuen/Desktop/DataScience/SAA/Asian Youth/')


athletes.to_csv('athletes_post_map_AY.csv', sep=',', encoding='utf-8-sig', index=False)


In [4092]:
# Load Asian Youth Benchmarks

os.chdir('/Users/veesheenyuen/Desktop/DataScience/SAA/Benchmarks/')

all_benchmark = pd.read_csv("All_Benchmarks.csv")




In [4093]:
all_benchmark

,BENCHMARK,EVENT,GENDER,RESULT
0,26th Asian Athletics,100m,Male,10.33
1,26th Asian Athletics,100m,Female,11.66
2,26th Asian Athletics,200m,Male,21.01
3,26th Asian Athletics,200m,Female,23.66
4,26th Asian Athletics,400m,Male,46.29
...,...,...,...,...
159,2025 Asian Youth,5000m Race Walk,Female,28:58.25
160,2025 Asian Youth,Decathlon,Male,5249.00
161,2025 Asian Youth,Heptathlon,Female,3965.00
162,2025 Asian Youth,Medley Relay,Male,01:59.16


In [4094]:
benchmark=all_benchmark[all_benchmark['BENCHMARK']=='2025 Asian Youth']

In [4095]:
benchmark

,BENCHMARK,EVENT,GENDER,RESULT
124,2025 Asian Youth,100m,Male,10.85
125,2025 Asian Youth,100m,Female,12.25
126,2025 Asian Youth,200m,Male,22.25
127,2025 Asian Youth,200m,Female,25.00
128,2025 Asian Youth,400m,Male,49.75
129,2025 Asian Youth,400m,Female,59.12
130,2025 Asian Youth,800m,Male,02:00.12
131,2025 Asian Youth,800m,Female,02:23.21
132,2025 Asian Youth,1500m,Male,04:08.48
133,2025 Asian Youth,1500m,Female,04:53.05


In [4096]:
benchmark=benchmark.reset_index(drop=True)

In [4097]:
# Converts any time format into seconds

def convert_time(i, string, metric):

    global output
    
    l=['discus', 'throw', 'jump', 'vault', 'shot']
        
    string=string.lower()
    
   # print('metric', metric)
    
    try:
        
        if 'w' in metric:  # skip marks with illegal wind speeds
            
        #    print('W', metric)
            
            output=''
            
        else:
            
    
            if any(s in string for s in l)==True:
            
                if 'm' in metric:
            
                    metric=metric.replace('m', '')
                    output=float(str(metric))
            
                elif 'GR' in metric:
            
                    metric=metric.replace('GR', '')
                    output=float(str(metric))
                
                
                else:
    
                    output=float(str(metric))
        
            elif string=='':   # no event description at all!
                
                output='' # return nothing
            
                
        
            else:
        
                searchstring = ":"
                searchstring2 = "."
                substring=str(metric)
                count = substring.count(searchstring)
                count2 = substring.count(searchstring2)
            
                if count==0:
                
                    output=float(substring)
            
            
                elif '10,000m' in string and count==2:  # fix erroneous timing format from XX:XX:XX to XX:XX.XX
                
                
                    idx = 5 # 6th character position
                    replacement = "."
                    metric = metric[:idx] + replacement + metric[idx+1:]                
                
                    m,s = metric.split(':')            

                    output = float(datetime.timedelta(minutes=int(m),seconds=float(s)).total_seconds())

                elif '5000m' in string and count==2:  # fix erroneous timing format from XX:XX:XX to XX:XX.XX
                
                
                    idx = 5 # 6th character position
                    replacement = "."
                    metric = metric[:idx] + replacement + metric[idx+1:]                
                
                    m,s = metric.split(':')            

                    output = float(datetime.timedelta(minutes=int(m),seconds=float(s)).total_seconds())

                    
                    
                elif '1500m' in string and count==2:  # fix erroneous timing format from XX:XX:XX to XX:XX.XX
                    
                    if len(substring)==7:  # format is X:XX:XX and not XX:XX:XX 
                        
                        idx = 4 # 5th character position
                        replacement = "."
                        metric = '0' + metric[:idx] + replacement + metric[idx+1:]                
                
                        m,s = metric.split(':')            

                        output = float(datetime.timedelta(minutes=int(m),seconds=float(s)).total_seconds())
                    
                        
                    else:  # format is XX:XX:XX
                        
                        idx = 5 # 5th character position
                        replacement = "."
                        metric = metric[:idx] + replacement + metric[idx+1:]                
                
                        m,s = metric.split(':')            

                        output = float(datetime.timedelta(minutes=int(m),seconds=float(s)).total_seconds())  
             
                elif (type(metric)==datetime.time or type(metric)==datetime.datetime):
                
                                                
                    time=str(metric)
                    h, m ,s = time.split(':')
                    output = float(datetime.timedelta(hours=int(h),minutes=int(m),seconds=float(s)).total_seconds())
            
                                
                elif (count==1 and count2==1):
            
                    m,s = metric.split(':')
                    output = float(datetime.timedelta(minutes=int(m),seconds=float(s)).total_seconds())
                     
                elif (count==1 and count2==2):
                
            
                    metric = metric.replace(".", ":", 1)
            
                    h,m,s = metric.split(':')            
                    output = float(datetime.timedelta(hours=int(h),minutes=int(m),seconds=float(s)).total_seconds())
                
        
                elif (count==2 and count2==0):
                
            
                    h,m,s = metric.split(':')
                    output = float(datetime.timedelta(hours=int(h),minutes=int(m),seconds=float(s)).total_seconds())
  
            

    except:
        
        pass
                
    return output

In [4098]:
def process_benchmarks(df):
    
    for i in range(len(df)):

        rowIndex = df.index[i]

        input_string=df.iloc[rowIndex,0]
    
        metric=df.iloc[rowIndex,3]
    
        if metric==None:
        
            continue
        
        out = convert_time(i, input_string, metric)
        
        print(rowIndex, input_string, out)

    
        df.loc[rowIndex, 'Metric'] = out
    
    return df

In [4099]:
process_benchmarks(benchmark)

0 2025 Asian Youth 10.85
1 2025 Asian Youth 12.25
2 2025 Asian Youth 22.25
3 2025 Asian Youth 25.0
4 2025 Asian Youth 49.75
5 2025 Asian Youth 59.12
6 2025 Asian Youth 120.12
7 2025 Asian Youth 143.21
8 2025 Asian Youth 248.48
9 2025 Asian Youth 293.05
10 2025 Asian Youth 551.87
11 2025 Asian Youth 656.29
12 2025 Asian Youth 386.83
13 2025 Asian Youth 468.16
14 2025 Asian Youth 14.25
15 2025 Asian Youth 15.53
16 2025 Asian Youth 56.39
17 2025 Asian Youth 67.36
18 2025 Asian Youth 1.97
19 2025 Asian Youth 1.62
20 2025 Asian Youth 6.94
21 2025 Asian Youth 5.57
22 2025 Asian Youth 13.63
23 2025 Asian Youth 11.88
24 2025 Asian Youth 16.69
25 2025 Asian Youth 12.74
26 2025 Asian Youth 49.94
27 2025 Asian Youth 33.99
28 2025 Asian Youth 52.57
29 2025 Asian Youth 33.6
30 2025 Asian Youth 58.78
31 2025 Asian Youth 40.61
32 2025 Asian Youth 4.03
33 2025 Asian Youth 3.08
34 2025 Asian Youth 2875.22
35 2025 Asian Youth 1738.25
36 2025 Asian Youth 5249.0
37 2025 Asian Youth 3965.0
38 2025 Asian Yo

,BENCHMARK,EVENT,GENDER,RESULT,Metric
0,2025 Asian Youth,100m,Male,10.85,10.85
1,2025 Asian Youth,100m,Female,12.25,12.25
2,2025 Asian Youth,200m,Male,22.25,22.25
3,2025 Asian Youth,200m,Female,25.00,25.00
4,2025 Asian Youth,400m,Male,49.75,49.75
5,2025 Asian Youth,400m,Female,59.12,59.12
6,2025 Asian Youth,800m,Male,02:00.12,120.12
7,2025 Asian Youth,800m,Female,02:23.21,143.21
8,2025 Asian Youth,1500m,Male,04:08.48,248.48
9,2025 Asian Youth,1500m,Female,04:53.05,293.05


In [4100]:
'''
for i in range(len(benchmarks)):
        
    rowIndex = benchmarks.index[i]

    input_string=benchmarks.iloc[rowIndex,0]
    
    metric=benchmarks.iloc[rowIndex,3]
    
    if metric==None:
        continue
        
    out = convert_time(i, input_string, metric)
    
    print(rowIndex, input_string, out)
     
    benchmarks.loc[rowIndex, 'Metric'] = out
'''

"\nfor i in range(len(benchmarks)):\n        \n    rowIndex = benchmarks.index[i]\n\n    input_string=benchmarks.iloc[rowIndex,0]\n    \n    metric=benchmarks.iloc[rowIndex,3]\n    \n    if metric==None:\n        continue\n        \n    out = convert_time(i, input_string, metric)\n    \n    print(rowIndex, input_string, out)\n     \n    benchmarks.loc[rowIndex, 'Metric'] = out\n"

In [4101]:
benchmark

,BENCHMARK,EVENT,GENDER,RESULT,Metric
0,2025 Asian Youth,100m,Male,10.85,10.85
1,2025 Asian Youth,100m,Female,12.25,12.25
2,2025 Asian Youth,200m,Male,22.25,22.25
3,2025 Asian Youth,200m,Female,25.00,25.00
4,2025 Asian Youth,400m,Male,49.75,49.75
5,2025 Asian Youth,400m,Female,59.12,59.12
6,2025 Asian Youth,800m,Male,02:00.12,120.12
7,2025 Asian Youth,800m,Female,02:23.21,143.21
8,2025 Asian Youth,1500m,Male,04:08.48,248.48
9,2025 Asian Youth,1500m,Female,04:53.05,293.05


In [4102]:
mask = benchmark['EVENT'].str.contains(r'jump|throw|Pole|put|Jump|Throw|pole|Put|Decathlon|Heptathlon', na=True)

benchmark.loc[mask, '2%']=benchmark['Metric']*0.98
benchmark.loc[mask, '3.5%']=benchmark['Metric']*0.965
benchmark.loc[mask, '5%']=benchmark['Metric']*0.95

benchmark.loc[~mask, '2%']=benchmark['Metric']*1.02
benchmark.loc[~mask, '3.5%']=benchmark['Metric']*1.035
benchmark.loc[~mask, '5%']=benchmark['Metric']*1.05


#benchmarks.iloc[5, [1]]='10000m run'
#benchmarks.iloc[28, [1]]='10000m run'
#benchmarks.iloc[26, [1]]='1500m'


In [4103]:
benchmark['MAPPED_EVENT']=benchmark['EVENT'].str.strip()

In [4104]:
for col in benchmark.columns:
    benchmark[col] = benchmark[col].astype(str)
    benchmark[col] = benchmark[col].str.replace('\xa0', ' ', regex=True)
    benchmark[col] = benchmark[col].str.replace('[\x00-\x1f\x7f-\x9f]', '', regex=True)
    benchmark[col] = benchmark[col].str.replace('\r', ' ', regex=True)
    benchmark[col] = benchmark[col].str.replace('\n', ' ', regex=True)
    benchmark[col] = benchmark[col].str.strip()


In [4105]:
benchmark

,BENCHMARK,EVENT,GENDER,RESULT,Metric,2%,3.5%,5%,MAPPED_EVENT
0,2025 Asian Youth,100m,Male,10.85,10.85,11.067,11.22975,11.3925,100m
1,2025 Asian Youth,100m,Female,12.25,12.25,12.495000000000001,12.678749999999999,12.8625,100m
2,2025 Asian Youth,200m,Male,22.25,22.25,22.695,23.02875,23.3625,200m
3,2025 Asian Youth,200m,Female,25.00,25.0,25.5,25.874999999999996,26.25,200m
4,2025 Asian Youth,400m,Male,49.75,49.75,50.745,51.491249999999994,52.237500000000004,400m
5,2025 Asian Youth,400m,Female,59.12,59.12,60.3024,61.18919999999999,62.076,400m
6,2025 Asian Youth,800m,Male,02:00.12,120.12,122.5224,124.32419999999999,126.126,800m
7,2025 Asian Youth,800m,Female,02:23.21,143.21,146.07420000000002,148.22235,150.37050000000002,800m
8,2025 Asian Youth,1500m,Male,04:08.48,248.48,253.4496,257.17679999999996,260.904,1500m
9,2025 Asian Youth,1500m,Female,04:53.05,293.05,298.911,303.30674999999997,307.70250000000004,1500m


In [4106]:
athletes

,NAME,RESULT,TEAM,AGE,COMPETITION_RANK,DIVISION,EVENT,DISTANCE,EVENT_CLASS,UNIQUE_ID,...,YEAR,REGION,TIMESTAMP,event_date,event_date_dt,delta_time,delta_time_conv,event_month,DOB_dt,MAPPED_EVENT
0,Zubin Percy Muncherji,01:54.0,None,None,2.0,None,800m,None,None,None,...,2025,International,2025-06-11 16:38:50.923703,8 June/25,2025-06-08,11 days 22:20:39.219731,11.0,6.0,1996-06-23 00:00:00,800m
1,Nicole Low Sui Xuan,18:34.34,None,None,6.0,None,5000m,None,None,None,...,2025,International,2025-06-11 16:38:50.923703,8 June/25,2025-06-08,11 days 22:20:39.219731,11.0,6.0,,5000m
2,Gabriel Lee,15.15,None,None,7.0,None,Triple Jump,None,None,None,...,2025,International,2025-06-11 16:38:50.923703,8 June/25,2025-06-08,11 days 22:20:39.219731,11.0,6.0,2003-02-23 00:00:00,Triple Jump
3,Esther Tay Shee Wei,3.2,None,None,10.0,None,Pole Vault,None,None,None,...,2025,International,2025-06-11 16:38:50.923703,8 June/25,2025-06-08,11 days 22:20:39.219731,11.0,6.0,,Pole Vault
4,Romaine Soh,02:16.90,None,None,5.0,None,800m,None,None,None,...,2025,International,2025-06-11 16:38:50.923703,8 June/25,2025-06-08,11 days 22:20:39.219731,11.0,6.0,1994-11-26 00:00:00,800m
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31154,St. Andrew's Junior College,03:43.06,SAJC,18.5,5.0,A,4x400m Relay,None,None,None,...,2025,Local,2025-06-06 15:35:30.807106,28-04-25,2025-04-28,52 days 22:20:39.219731,52.0,4.0,,4 x 400m
31155,Catholic Junior College,03:43.67,CJC,18.5,6.0,A,4x400m Relay,None,None,None,...,2025,Local,2025-06-06 15:35:30.807106,28-04-25,2025-04-28,52 days 22:20:39.219731,52.0,4.0,,4 x 400m
31156,Yishun Innova Junior College,03:45.03,YIJC,18.5,7.0,A,4x400m Relay,None,None,None,...,2025,Local,2025-06-06 15:35:30.807106,28-04-25,2025-04-28,52 days 22:20:39.219731,52.0,4.0,,4 x 400m
31157,National Junior College,03:47.40,NJC,18.5,8.0,A,4x400m Relay,None,None,None,...,2025,Local,2025-06-06 15:35:30.807106,28-04-25,2025-04-28,52 days 22:20:39.219731,52.0,4.0,,4 x 400m


In [4107]:
# There is a problem with RESULTS column being changed after this statement

#df = athletes.reset_index().merge(benchmarks.reset_index(), on=['MAPPED_EVENT','GENDER'], how='left')
#df = athletes.merge(benchmarks, on=['EVENT','GENDER'], how='left')


In [4108]:
# Merge benchmarks onto athletes on MAPPED_EVENT and GENDER

df = pd.merge(
    left=athletes, 
    right=benchmark,
    how='left',
    left_on=['MAPPED_EVENT', 'GENDER'],
    right_on=['MAPPED_EVENT', 'GENDER'],
)

In [4109]:
df

,NAME,RESULT_x,TEAM,AGE,COMPETITION_RANK,DIVISION,EVENT_x,DISTANCE,EVENT_CLASS,UNIQUE_ID,...,event_month,DOB_dt,MAPPED_EVENT,BENCHMARK,EVENT_y,RESULT_y,Metric,2%,3.5%,5%
0,Zubin Percy Muncherji,01:54.0,None,None,2.0,None,800m,None,None,None,...,6.0,1996-06-23 00:00:00,800m,2025 Asian Youth,800m,02:00.12,120.12,122.5224,124.32419999999999,126.126
1,Nicole Low Sui Xuan,18:34.34,None,None,6.0,None,5000m,None,None,None,...,6.0,,5000m,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Gabriel Lee,15.15,None,None,7.0,None,Triple Jump,None,None,None,...,6.0,2003-02-23 00:00:00,Triple Jump,2025 Asian Youth,Triple Jump,13.63,13.63,13.3574,13.15295,12.948500000000001
3,Esther Tay Shee Wei,3.2,None,None,10.0,None,Pole Vault,None,None,None,...,6.0,,Pole Vault,2025 Asian Youth,Pole Vault,3.08,3.08,3.0184,2.9722,2.9259999999999997
4,Romaine Soh,02:16.90,None,None,5.0,None,800m,None,None,None,...,6.0,1994-11-26 00:00:00,800m,2025 Asian Youth,800m,02:23.21,143.21,146.07420000000002,148.22235,150.37050000000002
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17328,St. Andrew's Junior College,03:43.06,SAJC,18.5,5.0,A,4x400m Relay,None,None,None,...,4.0,,4 x 400m,NaN,NaN,NaN,NaN,NaN,NaN,NaN
17329,Catholic Junior College,03:43.67,CJC,18.5,6.0,A,4x400m Relay,None,None,None,...,4.0,,4 x 400m,NaN,NaN,NaN,NaN,NaN,NaN,NaN
17330,Yishun Innova Junior College,03:45.03,YIJC,18.5,7.0,A,4x400m Relay,None,None,None,...,4.0,,4 x 400m,NaN,NaN,NaN,NaN,NaN,NaN,NaN
17331,National Junior College,03:47.40,NJC,18.5,8.0,A,4x400m Relay,None,None,None,...,4.0,,4 x 400m,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4110]:
# replace '-' with NaN

df['RESULT_x'] = df['RESULT_x'].replace(regex=r'–', value=np.NaN)
#df['SEED'] = df['SEED'].replace(regex=r'–', value=np.NaN)


In [4111]:
df

,NAME,RESULT_x,TEAM,AGE,COMPETITION_RANK,DIVISION,EVENT_x,DISTANCE,EVENT_CLASS,UNIQUE_ID,...,event_month,DOB_dt,MAPPED_EVENT,BENCHMARK,EVENT_y,RESULT_y,Metric,2%,3.5%,5%
0,Zubin Percy Muncherji,01:54.0,None,None,2.0,None,800m,None,None,None,...,6.0,1996-06-23 00:00:00,800m,2025 Asian Youth,800m,02:00.12,120.12,122.5224,124.32419999999999,126.126
1,Nicole Low Sui Xuan,18:34.34,None,None,6.0,None,5000m,None,None,None,...,6.0,,5000m,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Gabriel Lee,15.15,None,None,7.0,None,Triple Jump,None,None,None,...,6.0,2003-02-23 00:00:00,Triple Jump,2025 Asian Youth,Triple Jump,13.63,13.63,13.3574,13.15295,12.948500000000001
3,Esther Tay Shee Wei,3.2,None,None,10.0,None,Pole Vault,None,None,None,...,6.0,,Pole Vault,2025 Asian Youth,Pole Vault,3.08,3.08,3.0184,2.9722,2.9259999999999997
4,Romaine Soh,02:16.90,None,None,5.0,None,800m,None,None,None,...,6.0,1994-11-26 00:00:00,800m,2025 Asian Youth,800m,02:23.21,143.21,146.07420000000002,148.22235,150.37050000000002
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17328,St. Andrew's Junior College,03:43.06,SAJC,18.5,5.0,A,4x400m Relay,None,None,None,...,4.0,,4 x 400m,NaN,NaN,NaN,NaN,NaN,NaN,NaN
17329,Catholic Junior College,03:43.67,CJC,18.5,6.0,A,4x400m Relay,None,None,None,...,4.0,,4 x 400m,NaN,NaN,NaN,NaN,NaN,NaN,NaN
17330,Yishun Innova Junior College,03:45.03,YIJC,18.5,7.0,A,4x400m Relay,None,None,None,...,4.0,,4 x 400m,NaN,NaN,NaN,NaN,NaN,NaN,NaN
17331,National Junior College,03:47.40,NJC,18.5,8.0,A,4x400m Relay,None,None,None,...,4.0,,4 x 400m,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4112]:
os.chdir('/Users/veesheenyuen/Desktop/DataScience/SAA/Asian Youth/')


df.to_csv('Asian_Youth_postmap.csv', sep=',', encoding='utf-8-sig', index=False)


In [4113]:
# Convert results and seed into seconds format

df.reset_index(drop=True, inplace=True)

for col in df.columns:
    
    df[col] = df[col].astype(str)
    df[col] = df[col].str.replace('\xa0', ' ', regex=True)
    df[col] = df[col].str.replace('[\x00-\x1f\x7f-\x9f]', '', regex=True)
    df[col] = df[col].str.replace('\r', ' ', regex=True)
    df[col] = df[col].str.replace('\n', ' ', regex=True)
    df[col] = df[col].str.strip()

for i in range(len(df)):
    
    result_out=''
    
        
    rowIndex = df.index[i]

    event=df.loc[rowIndex,'MAPPED_EVENT']    # event description
    
    result=df.loc[rowIndex,'RESULT_x'] # result
    
    if result=='—' or result=='DQ' or result=='SCR' or result=='FS' or result=='DNQ' or result=='DNS' or result=='NH' or result=='NM' or result=='FOUL' or result=='DNF' or result=='SR' :
        continue
    
    result_out = convert_time(i, event, result)
         
    df.loc[rowIndex, 'RESULT_CONV'] = result_out



/var/folders/q5/yf8g5p896_b94gkbhqcjx3t40000gn/T/ipykernel_51640/201318515.py:30: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df.loc[rowIndex, 'RESULT_CONV'] = result_out


In [4114]:
df

,NAME,RESULT_x,TEAM,AGE,COMPETITION_RANK,DIVISION,EVENT_x,DISTANCE,EVENT_CLASS,UNIQUE_ID,...,DOB_dt,MAPPED_EVENT,BENCHMARK,EVENT_y,RESULT_y,Metric,2%,3.5%,5%,RESULT_CONV
0,Zubin Percy Muncherji,01:54.0,None,None,2.0,None,800m,None,None,None,...,1996-06-23 00:00:00,800m,2025 Asian Youth,800m,02:00.12,120.12,122.5224,124.32419999999999,126.126,114.0
1,Nicole Low Sui Xuan,18:34.34,None,None,6.0,None,5000m,None,None,None,...,,5000m,nan,nan,nan,nan,nan,nan,nan,1114.34
2,Gabriel Lee,15.15,None,None,7.0,None,Triple Jump,None,None,None,...,2003-02-23 00:00:00,Triple Jump,2025 Asian Youth,Triple Jump,13.63,13.63,13.3574,13.15295,12.948500000000001,15.15
3,Esther Tay Shee Wei,3.2,None,None,10.0,None,Pole Vault,None,None,None,...,,Pole Vault,2025 Asian Youth,Pole Vault,3.08,3.08,3.0184,2.9722,2.9259999999999997,3.2
4,Romaine Soh,02:16.90,None,None,5.0,None,800m,None,None,None,...,1994-11-26 00:00:00,800m,2025 Asian Youth,800m,02:23.21,143.21,146.07420000000002,148.22235,150.37050000000002,136.9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17328,St. Andrew's Junior College,03:43.06,SAJC,18.5,5.0,A,4x400m Relay,None,None,None,...,,4 x 400m,nan,nan,nan,nan,nan,nan,nan,223.06
17329,Catholic Junior College,03:43.67,CJC,18.5,6.0,A,4x400m Relay,None,None,None,...,,4 x 400m,nan,nan,nan,nan,nan,nan,nan,223.67
17330,Yishun Innova Junior College,03:45.03,YIJC,18.5,7.0,A,4x400m Relay,None,None,None,...,,4 x 400m,nan,nan,nan,nan,nan,nan,nan,225.03
17331,National Junior College,03:47.40,NJC,18.5,8.0,A,4x400m Relay,None,None,None,...,,4 x 400m,nan,nan,nan,nan,nan,nan,nan,227.4


In [4115]:
# Choose SEED if better than RESULT

#condition1=df['SEED_CONV']>df['RESULT_CONV']
#condition2=((df['CATEGORY_EVENT']=='Jump')|(df['CATEGORY_EVENT']=='Throw'))
#condition3=df['SEED_CONV']<df['RESULT_CONV']
#condition4=~((df['CATEGORY_EVENT']=='Jump')|(df['CATEGORY_EVENT']=='Throw'))


#df['RESULT_BEST']=df['SEED_CONV'].where((condition1 & condition2)|(condition3 & condition4), df['RESULT_CONV'].values)

df['RESULT_BEST'] = df['RESULT_CONV']

In [4116]:
df

,NAME,RESULT_x,TEAM,AGE,COMPETITION_RANK,DIVISION,EVENT_x,DISTANCE,EVENT_CLASS,UNIQUE_ID,...,MAPPED_EVENT,BENCHMARK,EVENT_y,RESULT_y,Metric,2%,3.5%,5%,RESULT_CONV,RESULT_BEST
0,Zubin Percy Muncherji,01:54.0,None,None,2.0,None,800m,None,None,None,...,800m,2025 Asian Youth,800m,02:00.12,120.12,122.5224,124.32419999999999,126.126,114.0,114.0
1,Nicole Low Sui Xuan,18:34.34,None,None,6.0,None,5000m,None,None,None,...,5000m,nan,nan,nan,nan,nan,nan,nan,1114.34,1114.34
2,Gabriel Lee,15.15,None,None,7.0,None,Triple Jump,None,None,None,...,Triple Jump,2025 Asian Youth,Triple Jump,13.63,13.63,13.3574,13.15295,12.948500000000001,15.15,15.15
3,Esther Tay Shee Wei,3.2,None,None,10.0,None,Pole Vault,None,None,None,...,Pole Vault,2025 Asian Youth,Pole Vault,3.08,3.08,3.0184,2.9722,2.9259999999999997,3.2,3.2
4,Romaine Soh,02:16.90,None,None,5.0,None,800m,None,None,None,...,800m,2025 Asian Youth,800m,02:23.21,143.21,146.07420000000002,148.22235,150.37050000000002,136.9,136.9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17328,St. Andrew's Junior College,03:43.06,SAJC,18.5,5.0,A,4x400m Relay,None,None,None,...,4 x 400m,nan,nan,nan,nan,nan,nan,nan,223.06,223.06
17329,Catholic Junior College,03:43.67,CJC,18.5,6.0,A,4x400m Relay,None,None,None,...,4 x 400m,nan,nan,nan,nan,nan,nan,nan,223.67,223.67
17330,Yishun Innova Junior College,03:45.03,YIJC,18.5,7.0,A,4x400m Relay,None,None,None,...,4 x 400m,nan,nan,nan,nan,nan,nan,nan,225.03,225.03
17331,National Junior College,03:47.40,NJC,18.5,8.0,A,4x400m Relay,None,None,None,...,4 x 400m,nan,nan,nan,nan,nan,nan,nan,227.4,227.4


In [4117]:
# Change to numeric

df[['2%', '3.5%', '5%', 'RESULT_BEST', 'Metric']] = df[['2%', '3.5%', '5%', 'RESULT_BEST', 'Metric']].apply(pd.to_numeric, errors='coerce')

In [4118]:
mask = df['CATEGORY_EVENT'].str.contains(r'Jump|Throw|jump|throw|Decathlon|Heptathlon|decathlon|heptathlon', na=True)

df.loc[mask, 'Delta2'] = df['RESULT_BEST']-df['2%']
df.loc[mask, 'Delta3.5'] = df['RESULT_BEST']-df['3.5%']
df.loc[mask, 'Delta5'] = df['RESULT_BEST']-df['5%']
df.loc[mask, 'Delta_Benchmark'] = df['RESULT_BEST']-df['Metric']

df.loc[~mask, 'Delta2'] =  df['2%'] - df['RESULT_BEST']
df.loc[~mask, 'Delta3.5'] = df['3.5%'] - df['RESULT_BEST']
df.loc[~mask, 'Delta5'] = df['5%'] - df['RESULT_BEST']
df.loc[~mask, 'Delta_Benchmark'] = df['Metric'] - df['RESULT_BEST']

#rslt_df['Delta2']=rslt_df['2pc']-rslt_df['RESULT_CONV']
#rslt_df['Delta35']=rslt_df['35pc']-rslt_df['RESULT_CONV']
#rslt_df['Delta5']=rslt_df['5pc']-rslt_df['RESULT_CONV']
df=df.loc[df['COMPETITION']!='Southeast Asian Games'] # Do not include results from SEAG in dataset

In [4119]:
# Performance metric to filter out athletes

df['PERF_SCALAR']=df['Delta5']/df['Metric']*100

In [4120]:
os.chdir('/Users/veesheenyuen/Desktop/DataScience/SAA/Asian Youth/')


df.to_csv('AY_postmap_benchmarked.csv', sep=',', encoding='utf-8-sig', index=False)


In [4121]:
df

,NAME,RESULT_x,TEAM,AGE,COMPETITION_RANK,DIVISION,EVENT_x,DISTANCE,EVENT_CLASS,UNIQUE_ID,...,2%,3.5%,5%,RESULT_CONV,RESULT_BEST,Delta2,Delta3.5,Delta5,Delta_Benchmark,PERF_SCALAR
0,Zubin Percy Muncherji,01:54.0,None,None,2.0,None,800m,None,None,None,...,122.5224,124.32420,126.1260,114.0,114.00,8.5224,10.32420,12.1260,6.12,10.094905
1,Nicole Low Sui Xuan,18:34.34,None,None,6.0,None,5000m,None,None,None,...,NaN,NaN,NaN,1114.34,1114.34,NaN,NaN,NaN,NaN,NaN
2,Gabriel Lee,15.15,None,None,7.0,None,Triple Jump,None,None,None,...,13.3574,13.15295,12.9485,15.15,15.15,1.7926,1.99705,2.2015,1.52,16.151871
3,Esther Tay Shee Wei,3.2,None,None,10.0,None,Pole Vault,None,None,None,...,3.0184,2.97220,2.9260,3.2,3.20,0.1816,0.22780,0.2740,0.12,8.896104
4,Romaine Soh,02:16.90,None,None,5.0,None,800m,None,None,None,...,146.0742,148.22235,150.3705,136.9,136.90,9.1742,11.32235,13.4705,6.31,9.406117
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17328,St. Andrew's Junior College,03:43.06,SAJC,18.5,5.0,A,4x400m Relay,None,None,None,...,NaN,NaN,NaN,223.06,223.06,NaN,NaN,NaN,NaN,NaN
17329,Catholic Junior College,03:43.67,CJC,18.5,6.0,A,4x400m Relay,None,None,None,...,NaN,NaN,NaN,223.67,223.67,NaN,NaN,NaN,NaN,NaN
17330,Yishun Innova Junior College,03:45.03,YIJC,18.5,7.0,A,4x400m Relay,None,None,None,...,NaN,NaN,NaN,225.03,225.03,NaN,NaN,NaN,NaN,NaN
17331,National Junior College,03:47.40,NJC,18.5,8.0,A,4x400m Relay,None,None,None,...,NaN,NaN,NaN,227.4,227.40,NaN,NaN,NaN,NaN,NaN


In [4122]:
df['DOB_dt'].head(50)

0     1996-06-23 00:00:00
1                        
2     2003-02-23 00:00:00
3                        
4     1994-11-26 00:00:00
5     1994-11-26 00:00:00
6     1998-02-23 00:00:00
7     2000-10-14 00:00:00
8     2003-09-23 00:00:00
9     1996-09-20 00:00:00
10    1996-09-20 00:00:00
11    1994-07-03 00:00:00
12                       
13    2001-12-18 00:00:00
14    2003-09-23 00:00:00
15    2003-09-23 00:00:00
16    2003-09-23 00:00:00
17    1994-07-03 00:00:00
18    1996-02-26 00:00:00
19                       
20    1996-09-20 00:00:00
21    2004-01-06 00:00:00
22    2002-08-07 00:00:00
23    2002-08-07 00:00:00
24    2000-10-28 00:00:00
25    1997-01-12 00:00:00
26    1994-07-03 00:00:00
27    1996-02-26 00:00:00
28                       
29                       
30    1996-09-20 00:00:00
31    1996-09-20 00:00:00
32    1996-09-20 00:00:00
33    2006-01-22 00:00:00
34    1997-06-08 00:00:00
35    1997-06-08 00:00:00
36    2001-03-06 00:00:00
37                       
38    2004-0

In [4123]:
# Check that there are no empty DOB fields in dob_df
# Filter out empty DOB fields to build dictionary

#dob_df = df[((str(df['DOB_dt'])!='NaT')|(df['DOB_dt']!='None')|(df['DOB_dt'].notnull()))]
#dob_df = df.dropna(subset=['DOB_dt'])
#dob_df = df[df['DOB_dt']!='']
dob_df = df[df.DOB_dt.apply(lambda x: len(str(x))>2)]


os.chdir('/Users/veesheenyuen/Desktop/DataScience/SAA/Asian Youth/')

dob_df.to_csv('dob_df.csv', encoding='utf-8')

dob_df

,NAME,RESULT_x,TEAM,AGE,COMPETITION_RANK,DIVISION,EVENT_x,DISTANCE,EVENT_CLASS,UNIQUE_ID,...,2%,3.5%,5%,RESULT_CONV,RESULT_BEST,Delta2,Delta3.5,Delta5,Delta_Benchmark,PERF_SCALAR
0,Zubin Percy Muncherji,01:54.0,None,None,2.0,None,800m,None,None,None,...,122.5224,124.32420,126.1260,114.0,114.00,8.5224,10.32420,12.1260,6.12,10.094905
2,Gabriel Lee,15.15,None,None,7.0,None,Triple Jump,None,None,None,...,13.3574,13.15295,12.9485,15.15,15.15,1.7926,1.99705,2.2015,1.52,16.151871
4,Romaine Soh,02:16.90,None,None,5.0,None,800m,None,None,None,...,146.0742,148.22235,150.3705,136.9,136.90,9.1742,11.32235,13.4705,6.31,9.406117
5,Romaine Soh,04:39.9,None,None,5.0,None,1500m,None,None,None,...,298.9110,303.30675,307.7025,279.9,279.90,19.0110,23.40675,27.8025,13.15,9.487289
6,Vanessa Lee Ying Zhuang,17:29.5,None,None,3.0,None,5000m,None,None,None,...,NaN,NaN,NaN,1049.5,1049.50,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12837,"TEO, MARK XAVIER",12.02m,Catholic High School,15,5,U18,Triple Jump,0,None,M391I10,...,13.3574,13.15295,12.9485,12.02,12.02,-1.3374,-1.13295,-0.9285,-1.61,-6.812179
12838,"SETHO, BEATRICE",9.76m,Nanyang Girls' High School,14,2,U15,Triple Jump,0,None,B332H11,...,11.6424,11.46420,11.2860,9.76,9.76,-1.8824,-1.70420,-1.5260,-2.12,-12.845118
12839,"Lau, Andrew",12.26m,Raffles Institution,15,5,U18,Triple Jump,0,None,A138H10,...,13.3574,13.15295,12.9485,12.26,12.26,-1.0974,-0.89295,-0.6885,-1.37,-5.051357
12840,"Yen, Ming Zhen",12.23m,National Junior College,17,4,U20,Triple Jump,0,None,M034B08,...,13.3574,13.15295,12.9485,12.23,12.23,-1.1274,-0.92295,-0.7185,-1.40,-5.271460


In [4124]:
# Create dictionary of NAME, DOB for everyone with a valid DOB
# Must map the name variation to a name from the df with all null DOB filtered out

dictionary_dob = dict(zip(dob_df['NAME'].str.casefold(), dob_df['DOB_dt']))

dictionary_dob

{'zubin percy muncherji': '1996-06-23 00:00:00',
 'gabriel lee': '2003-02-23 00:00:00',
 'romaine soh': '1994-11-26 00:00:00',
 'vanessa lee ying zhuang': '1998-02-23 00:00:00',
 'tia louise rozario': '2000-10-14 00:00:00',
 'elizabeth-ann tan': '2003-09-23 00:00:00',
 'shanti veronica pereira': '1996-09-20 00:00:00',
 'chen xiang ang': '1994-07-03 00:00:00',
 'tan daryl': '2001-12-18 00:00:00',
 'jun jie calvin quek': '1996-02-26 00:00:00',
 'jamie el-red ang': '2004-01-06 00:00:00',
 'marc brian louis': '2002-08-07 00:00:00',
 'rajan thiruben thana': '2000-10-28 00:00:00',
 'shaun goh': '1997-01-12 00:00:00',
 'laavina jaganith': '2006-01-22 00:00:00',
 'kerstin ong jing rong': '1997-06-08 00:00:00',
 'kampton kam': '2001-03-06 00:00:00',
 'joshua chua': '2004-02-25 00:00:00',
 'lucas le cong fun': '2006-01-11 00:00:00',
 'harry irfan curra': '2008-06-19 00:00:00',
 'yee chern clara lim': '2006-06-19 00:00:00',
 'ian koe': '1999-05-04 00:00:00',
 'raeshon loo wei ming': '2001-01-31 0

In [4125]:
import string

# Create dictionary version with punctuation stripped out

translator = str.maketrans('', '', string.punctuation)
dob_df['clean_name'] = dob_df['NAME'].str.translate(translator)
dob_df['clean_name'] = dob_df['clean_name'].str.casefold()

/var/folders/q5/yf8g5p896_b94gkbhqcjx3t40000gn/T/ipykernel_51640/4017585124.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dob_df['clean_name'] = dob_df['NAME'].str.translate(translator)
/var/folders/q5/yf8g5p896_b94gkbhqcjx3t40000gn/T/ipykernel_51640/4017585124.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dob_df['clean_name'] = dob_df['clean_name'].str.casefold()


In [4126]:
dob_df

,NAME,RESULT_x,TEAM,AGE,COMPETITION_RANK,DIVISION,EVENT_x,DISTANCE,EVENT_CLASS,UNIQUE_ID,...,3.5%,5%,RESULT_CONV,RESULT_BEST,Delta2,Delta3.5,Delta5,Delta_Benchmark,PERF_SCALAR,clean_name
0,Zubin Percy Muncherji,01:54.0,None,None,2.0,None,800m,None,None,None,...,124.32420,126.1260,114.0,114.00,8.5224,10.32420,12.1260,6.12,10.094905,zubin percy muncherji
2,Gabriel Lee,15.15,None,None,7.0,None,Triple Jump,None,None,None,...,13.15295,12.9485,15.15,15.15,1.7926,1.99705,2.2015,1.52,16.151871,gabriel lee
4,Romaine Soh,02:16.90,None,None,5.0,None,800m,None,None,None,...,148.22235,150.3705,136.9,136.90,9.1742,11.32235,13.4705,6.31,9.406117,romaine soh
5,Romaine Soh,04:39.9,None,None,5.0,None,1500m,None,None,None,...,303.30675,307.7025,279.9,279.90,19.0110,23.40675,27.8025,13.15,9.487289,romaine soh
6,Vanessa Lee Ying Zhuang,17:29.5,None,None,3.0,None,5000m,None,None,None,...,NaN,NaN,1049.5,1049.50,NaN,NaN,NaN,NaN,NaN,vanessa lee ying zhuang
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12837,"TEO, MARK XAVIER",12.02m,Catholic High School,15,5,U18,Triple Jump,0,None,M391I10,...,13.15295,12.9485,12.02,12.02,-1.3374,-1.13295,-0.9285,-1.61,-6.812179,teo mark xavier
12838,"SETHO, BEATRICE",9.76m,Nanyang Girls' High School,14,2,U15,Triple Jump,0,None,B332H11,...,11.46420,11.2860,9.76,9.76,-1.8824,-1.70420,-1.5260,-2.12,-12.845118,setho beatrice
12839,"Lau, Andrew",12.26m,Raffles Institution,15,5,U18,Triple Jump,0,None,A138H10,...,13.15295,12.9485,12.26,12.26,-1.0974,-0.89295,-0.6885,-1.37,-5.051357,lau andrew
12840,"Yen, Ming Zhen",12.23m,National Junior College,17,4,U20,Triple Jump,0,None,M034B08,...,13.15295,12.9485,12.23,12.23,-1.1274,-0.92295,-0.7185,-1.40,-5.271460,yen ming zhen


In [4127]:
# Dictionary without punctuation and capital letters

dictionary_dob_clean_name = dict(zip(dob_df['clean_name'].str.casefold(), dob_df['DOB_dt']))


In [4128]:
dictionary_dob_clean_name

{'zubin percy muncherji': '1996-06-23 00:00:00',
 'gabriel lee': '2003-02-23 00:00:00',
 'romaine soh': '1994-11-26 00:00:00',
 'vanessa lee ying zhuang': '1998-02-23 00:00:00',
 'tia louise rozario': '2000-10-14 00:00:00',
 'elizabethann tan': '2003-09-23 00:00:00',
 'shanti veronica pereira': '1996-09-20 00:00:00',
 'chen xiang ang': '1994-07-03 00:00:00',
 'tan daryl': '2001-12-18 00:00:00',
 'jun jie calvin quek': '1996-02-26 00:00:00',
 'jamie elred ang': '2004-01-06 00:00:00',
 'marc brian louis': '2002-08-07 00:00:00',
 'rajan thiruben thana': '2000-10-28 00:00:00',
 'shaun goh': '1997-01-12 00:00:00',
 'laavina jaganith': '2006-01-22 00:00:00',
 'kerstin ong jing rong': '1997-06-08 00:00:00',
 'kampton kam': '2001-03-06 00:00:00',
 'joshua chua': '2004-02-25 00:00:00',
 'lucas le cong fun': '2006-01-11 00:00:00',
 'harry irfan curra': '2008-06-19 00:00:00',
 'yee chern clara lim': '2006-06-19 00:00:00',
 'ian koe': '1999-05-04 00:00:00',
 'raeshon loo wei ming': '2001-01-31 00:

In [4129]:
len(dictionary_dob)

5323

In [4130]:
dictionary_dob_clean_name.get('nitin pranit')

'2007-09-18 00:00:00'

In [4131]:
'''
# Read a variation name list and corrections from CSVs

os.chdir('/Users/veesheenyuen/Desktop/DataScience/SAA/Name Variations/')

names = pd.read_csv("junior_names.csv")

for index, row in names.iterrows():
        
    print(names.VARIATION, names.NAME)
    df['NAME_MAPPED'] = df['NAME'].replace(regex=rf"{row['VARIATION']}", value=f"{row['NAME']}")
'''

'\n# Read a variation name list and corrections from CSVs\n\nos.chdir(\'/Users/veesheenyuen/Desktop/DataScience/SAA/Name Variations/\')\n\nnames = pd.read_csv("junior_names.csv")\n\nfor index, row in names.iterrows():\n        \n    print(names.VARIATION, names.NAME)\n    df[\'NAME_MAPPED\'] = df[\'NAME\'].replace(regex=rf"{row[\'VARIATION\']}", value=f"{row[\'NAME\']}")\n'

In [4132]:
# Read name variations from GCS name lists bucket (Still in beta)

df['NAME'] = df['NAME'].str.replace('\xa0', '', regex=True)
df['NAME'] = df['NAME'].str.replace('[\x00-\x1f\x7f-\x9f]', '', regex=True)
df['NAME'] = df['NAME'].str.replace('\r', '', regex=True)
df['NAME'] = df['NAME'].str.replace('\n', '', regex=True)
df['NAME'] = df['NAME'].str.strip()

df['NAME'] = df['NAME'].str.casefold()  # everything lower case


# Read csv from GCS bucket

file_path = "gs://name_variations/name_variations.csv"
names = pd.read_csv(file_path,
                 sep=",",
                 storage_options={"token": '/Users/veesheenyuen/Desktop/DataScience/Keys/saa-analytics-7c8937b70609.json'})


# Clean and apply casefold to name variations

names['NAME'] = names['NAME'].str.replace('\xa0', '', regex=True)
names['NAME'] = names['NAME'].str.replace('[\x00-\x1f\x7f-\x9f]', '', regex=True)
names['NAME'] = names['NAME'].str.replace('\r', '', regex=True)
names['NAME'] = names['NAME'].str.replace('\n', '', regex=True)
names['NAME'] = names['NAME'].str.strip()
names['VARIATION'] = names['VARIATION'].str.replace('\xa0', '', regex=True)
names['VARIATION'] = names['VARIATION'].str.replace('[\x00-\x1f\x7f-\x9f]', '', regex=True)
names['VARIATION'] = names['VARIATION'].str.replace('\r', '', regex=True)
names['VARIATION'] = names['VARIATION'].str.replace('\n', '', regex=True)
names['VARIATION'] = names['VARIATION'].str.strip()

names['VARIATION'] = names['VARIATION'].str.casefold() # convert to lower case
names['NAME'] = names['NAME'].str.casefold()


# Iterate over dataframe and replace names

for index, row in names.iterrows():
        
    df['NAME'] = df['NAME'].replace(regex=rf"{row['VARIATION']}", value=f"{row['NAME']}")
    

    
df['NAME'] = df['NAME'].str.title()  # capitalize first letter


In [4133]:
os.chdir('/Users/veesheenyuen/Desktop/DataScience/SAA/Asian Youth/')


df.to_csv("names_check.csv", encoding='utf-8')

In [4134]:
# Exclude foreigners from MALAYSIA, THAILAND etc.

#df_select = df[(df['TEAM']!='Malaysia') & (df['TEAM']!='THAILAND') & (df['TEAM']!='China') & (df['TEAM']!='South Korea') & (df['TEAM']!='Laos') & (df['TEAM']!='Philippines') & (df['TEAM']!='Piboonbumpen Thailand') & (df['TEAM']!='Chinese Taipei') & (df['TEAM']!='Gurkha Contingent') & (df['TEAM']!='Australia') & (df['TEAM']!='Piboonbumpen Thailand') & (df['TEAM']!='Hong Kong') & (df['TEAM']!='PERAK')] 

df_select = df[(df['TEAM']!='Malaysia')&(df['TEAM']!='THAILAND')&(df['TEAM']!='China')&(df['TEAM']!='Thailand') 
                       &(df['TEAM']!='South Korea')&(df['TEAM']!='Laos')&(df['TEAM']!='Myanmar') 
                       &(df['TEAM']!='Philippines')&(df['TEAM']!='Piboonbumpen Thailand') 
                       &(df['TEAM']!='Chinese Taipei')&(df['TEAM']!='Gurkha Contingent') 
                       &(df['TEAM']!='Australia')&(df['TEAM']!='Piboonbumpen Thailand') 
                       &(df['TEAM']!='Hong Kong')&(df['TEAM']!='PERAK')&(df['TEAM']!='Sri Lanka') 
                       &(df['TEAM']!='Indonesia')&(df['TEAM']!='THAILAND')&(df['TEAM']!='MALAYSIA') 
                       &(df['TEAM']!='PHILIPPINES') & (df['TEAM']!='SOUTH KOREA')&(df['TEAM']!='Waseda') 
                       &(df['TEAM']!='LAOS')&(df['TEAM']!='CHINESE TAIPEI')&(df['TEAM']!='Vietnam')
                       &(df['TEAM']!='INDIA')&(df['TEAM']!='Hong Kong, China')&(df['TEAM']!='AIC JAPAN')] 

In [4135]:
df_select

,NAME,RESULT_x,TEAM,AGE,COMPETITION_RANK,DIVISION,EVENT_x,DISTANCE,EVENT_CLASS,UNIQUE_ID,...,2%,3.5%,5%,RESULT_CONV,RESULT_BEST,Delta2,Delta3.5,Delta5,Delta_Benchmark,PERF_SCALAR
0,Zubin Percy Muncherji,01:54.0,None,None,2.0,None,800m,None,None,None,...,122.5224,124.32420,126.1260,114.0,114.00,8.5224,10.32420,12.1260,6.12,10.094905
1,"Low, Nicole Sui Xuan",18:34.34,None,None,6.0,None,5000m,None,None,None,...,NaN,NaN,NaN,1114.34,1114.34,NaN,NaN,NaN,NaN,NaN
2,Lee Gabriel Jin Yi,15.15,None,None,7.0,None,Triple Jump,None,None,None,...,13.3574,13.15295,12.9485,15.15,15.15,1.7926,1.99705,2.2015,1.52,16.151871
3,"Tay, Esther",3.2,None,None,10.0,None,Pole Vault,None,None,None,...,3.0184,2.97220,2.9260,3.2,3.20,0.1816,0.22780,0.2740,0.12,8.896104
4,Romaine Soh Rui Min,02:16.90,None,None,5.0,None,800m,None,None,None,...,146.0742,148.22235,150.3705,136.9,136.90,9.1742,11.32235,13.4705,6.31,9.406117
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17328,St. Andrew'S Junior College,03:43.06,SAJC,18.5,5.0,A,4x400m Relay,None,None,None,...,NaN,NaN,NaN,223.06,223.06,NaN,NaN,NaN,NaN,NaN
17329,Catholic Junior College,03:43.67,CJC,18.5,6.0,A,4x400m Relay,None,None,None,...,NaN,NaN,NaN,223.67,223.67,NaN,NaN,NaN,NaN,NaN
17330,Yishun Innova Junior College,03:45.03,YIJC,18.5,7.0,A,4x400m Relay,None,None,None,...,NaN,NaN,NaN,225.03,225.03,NaN,NaN,NaN,NaN,NaN
17331,National Junior College,03:47.40,NJC,18.5,8.0,A,4x400m Relay,None,None,None,...,NaN,NaN,NaN,227.4,227.40,NaN,NaN,NaN,NaN,NaN


In [4136]:
# Remove spurious SEED results 

#df_select=df_select[~((df_select['NAME']=='Tan, Kian Ye') & (df_select['MAPPED_EVENT']=='200m'))]
#df_select=df_select[~((df_select['NAME']=='LOH, BEAVERLY') & (df_select['MAPPED_EVENT']=='400m'))]
#df_select=df_select[~((df_select['NAME']=='Vaz, Luisa Marie') & (df_select['MAPPED_EVENT']=='High jump'))]


In [4137]:
df_select

,NAME,RESULT_x,TEAM,AGE,COMPETITION_RANK,DIVISION,EVENT_x,DISTANCE,EVENT_CLASS,UNIQUE_ID,...,2%,3.5%,5%,RESULT_CONV,RESULT_BEST,Delta2,Delta3.5,Delta5,Delta_Benchmark,PERF_SCALAR
0,Zubin Percy Muncherji,01:54.0,None,None,2.0,None,800m,None,None,None,...,122.5224,124.32420,126.1260,114.0,114.00,8.5224,10.32420,12.1260,6.12,10.094905
1,"Low, Nicole Sui Xuan",18:34.34,None,None,6.0,None,5000m,None,None,None,...,NaN,NaN,NaN,1114.34,1114.34,NaN,NaN,NaN,NaN,NaN
2,Lee Gabriel Jin Yi,15.15,None,None,7.0,None,Triple Jump,None,None,None,...,13.3574,13.15295,12.9485,15.15,15.15,1.7926,1.99705,2.2015,1.52,16.151871
3,"Tay, Esther",3.2,None,None,10.0,None,Pole Vault,None,None,None,...,3.0184,2.97220,2.9260,3.2,3.20,0.1816,0.22780,0.2740,0.12,8.896104
4,Romaine Soh Rui Min,02:16.90,None,None,5.0,None,800m,None,None,None,...,146.0742,148.22235,150.3705,136.9,136.90,9.1742,11.32235,13.4705,6.31,9.406117
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17328,St. Andrew'S Junior College,03:43.06,SAJC,18.5,5.0,A,4x400m Relay,None,None,None,...,NaN,NaN,NaN,223.06,223.06,NaN,NaN,NaN,NaN,NaN
17329,Catholic Junior College,03:43.67,CJC,18.5,6.0,A,4x400m Relay,None,None,None,...,NaN,NaN,NaN,223.67,223.67,NaN,NaN,NaN,NaN,NaN
17330,Yishun Innova Junior College,03:45.03,YIJC,18.5,7.0,A,4x400m Relay,None,None,None,...,NaN,NaN,NaN,225.03,225.03,NaN,NaN,NaN,NaN,NaN
17331,National Junior College,03:47.40,NJC,18.5,8.0,A,4x400m Relay,None,None,None,...,NaN,NaN,NaN,227.4,227.40,NaN,NaN,NaN,NaN,NaN


In [4138]:
'''
# Read list of foreigners

foreigners = pd.read_csv('/Users/veesheenyuen/Desktop/DataScience/SAA/MM/List of Foreigners.csv', encoding='latin-1')
'''


"\n# Read list of foreigners\n\nforeigners = pd.read_csv('/Users/veesheenyuen/Desktop/DataScience/SAA/MM/List of Foreigners.csv', encoding='latin-1')\n"

In [4139]:
# Read list of foreigners from GCS bucket

file_path = "gs://name_lists/List of Foreigners.csv"
foreigners = pd.read_csv(file_path,
                 sep=",",
                 encoding="unicode escape",
                 storage_options={"token": '/Users/veesheenyuen/Desktop/DataScience/Keys/saa-analytics-7c8937b70609.json'})


In [4140]:
foreigners

,LAST_NAME,FIRST_NAME
0,Aaryan,Greuter Christoph
1,Akahodani,Takayuki
2,Apondar,Audric
3,Brooks,Ruby
4,Brouwer,Cees
...,...,...
235,Kashama,Biwesa Daniel
236,ISMAIL,MUHAMMAD ZULFIQAR
237,Jayaganeson,Kirtisha
238,LIN,Yu Sian


In [4141]:
foreigners['V1'] = foreigners['LAST_NAME']+' '+foreigners['FIRST_NAME']
foreigners['V2'] = foreigners['FIRST_NAME']+' '+foreigners['LAST_NAME']
foreigners['V3'] = foreigners['LAST_NAME']+', '+foreigners['FIRST_NAME']
foreigners['V4'] = foreigners['FIRST_NAME']+' '+foreigners['LAST_NAME']

for1 = foreigners['V1'].dropna().tolist()
for2 = foreigners['V2'].dropna().tolist()
for3 = foreigners['V3'].dropna().tolist()
for4 = foreigners['V4'].dropna().tolist()

foreign_list = for1+for2+for3+for4 

foreign_list_casefold=[s.casefold() for s in foreign_list]

exclusions = foreign_list_casefold

no_foreigners_list = df_select.loc[~df['NAME'].str.casefold().isin(exclusions)]  # ~ means NOT IN. DROP spex carded athletes

In [4142]:
no_foreigners_list

,NAME,RESULT_x,TEAM,AGE,COMPETITION_RANK,DIVISION,EVENT_x,DISTANCE,EVENT_CLASS,UNIQUE_ID,...,2%,3.5%,5%,RESULT_CONV,RESULT_BEST,Delta2,Delta3.5,Delta5,Delta_Benchmark,PERF_SCALAR
0,Zubin Percy Muncherji,01:54.0,None,None,2.0,None,800m,None,None,None,...,122.5224,124.32420,126.1260,114.0,114.00,8.5224,10.32420,12.1260,6.12,10.094905
1,"Low, Nicole Sui Xuan",18:34.34,None,None,6.0,None,5000m,None,None,None,...,NaN,NaN,NaN,1114.34,1114.34,NaN,NaN,NaN,NaN,NaN
2,Lee Gabriel Jin Yi,15.15,None,None,7.0,None,Triple Jump,None,None,None,...,13.3574,13.15295,12.9485,15.15,15.15,1.7926,1.99705,2.2015,1.52,16.151871
3,"Tay, Esther",3.2,None,None,10.0,None,Pole Vault,None,None,None,...,3.0184,2.97220,2.9260,3.2,3.20,0.1816,0.22780,0.2740,0.12,8.896104
4,Romaine Soh Rui Min,02:16.90,None,None,5.0,None,800m,None,None,None,...,146.0742,148.22235,150.3705,136.9,136.90,9.1742,11.32235,13.4705,6.31,9.406117
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17328,St. Andrew'S Junior College,03:43.06,SAJC,18.5,5.0,A,4x400m Relay,None,None,None,...,NaN,NaN,NaN,223.06,223.06,NaN,NaN,NaN,NaN,NaN
17329,Catholic Junior College,03:43.67,CJC,18.5,6.0,A,4x400m Relay,None,None,None,...,NaN,NaN,NaN,223.67,223.67,NaN,NaN,NaN,NaN,NaN
17330,Yishun Innova Junior College,03:45.03,YIJC,18.5,7.0,A,4x400m Relay,None,None,None,...,NaN,NaN,NaN,225.03,225.03,NaN,NaN,NaN,NaN,NaN
17331,National Junior College,03:47.40,NJC,18.5,8.0,A,4x400m Relay,None,None,None,...,NaN,NaN,NaN,227.4,227.40,NaN,NaN,NaN,NaN,NaN


In [4143]:
# Choose the best result for each event participated by every athlete

top_performers_clean = no_foreigners_list.sort_values(['MAPPED_EVENT', 'NAME','PERF_SCALAR'],ascending=False).groupby(['MAPPED_EVENT', 'NAME']).head(1)


In [4144]:
top_performers_clean.reset_index(inplace=True)


In [4149]:
os.chdir('/Users/veesheenyuen/Desktop/DataScience/SAA/Asian Youth/')


top_performers_clean.to_csv('AY_top_performers_prod.csv', encoding='utf-8')

In [4150]:
top_performers_clean

,index,NAME,RESULT_x,TEAM,AGE,COMPETITION_RANK,DIVISION,EVENT_x,DISTANCE,EVENT_CLASS,...,2%,3.5%,5%,RESULT_CONV,RESULT_BEST,Delta2,Delta3.5,Delta5,Delta_Benchmark,PERF_SCALAR
0,16545,Zhou Xuanyu,9.12,DHS,13.5,10.0,C,Triple Jump,None,None,...,11.6424,11.46420,11.2860,9.12,9.12,-2.5224,-2.34420,-2.1660,-2.76,-18.232323
1,11022,"Zhong, Chuhan",12.26,None,None,None,None,Triple Jump,None,None,...,11.6424,11.46420,11.2860,12.26,12.26,0.6176,0.79580,0.9740,0.38,8.198653
2,1076,"Zheng, Justin De",10.47m,National Junior College,14,12,U15,Triple Jump,0,None,...,13.3574,13.15295,12.9485,10.47,10.47,-2.8874,-2.68295,-2.4785,-3.16,-18.184153
3,12853,Zheng Justin De,10.29,NJC,13.5,12.0,C,Triple Jump,None,None,...,13.3574,13.15295,12.9485,10.29,10.29,-3.0674,-2.86295,-2.6585,-3.34,-19.504769
4,2326,"Zhao, Daniel",11.67m,Hwa Chong Institution,14,3,U15,Triple Jump,0,None,...,13.3574,13.15295,12.9485,11.67,11.67,-1.6874,-1.48295,-1.2785,-1.96,-9.380044
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11356,3879,"., Shaik Isa",15.49,Club ZOOM,8,32,U9,Dash,80,,...,NaN,NaN,NaN,,NaN,NaN,NaN,NaN,NaN,NaN
11357,7899,"., Nur Elena",16.42,UNA,6,1,U7,Dash,80,,...,NaN,NaN,NaN,,NaN,NaN,NaN,NaN,NaN,NaN
11358,9907,"., Dharkshitha",08:35.6,Cedar Girls Secondary School,15,1,U18,Race Walk,1500,None,...,NaN,NaN,NaN,,NaN,NaN,NaN,NaN,NaN,NaN
11359,3520,"., Cayden",06:05.1,North Vista,12,54,Open,Mile Run,1,,...,NaN,NaN,NaN,,NaN,NaN,NaN,NaN,NaN,NaN


In [4151]:
# Choose best performance for each event

#tiered_performers = top_performers_clean.sort_values(['GENDER', 'MAPPED_EVENT', 'PERF_SCALAR'],ascending=False).groupby(['MAPPED_EVENT', 'NAME']).head(1)

tiered_performers = top_performers_clean


In [4152]:
tiered_performers

,index,NAME,RESULT_x,TEAM,AGE,COMPETITION_RANK,DIVISION,EVENT_x,DISTANCE,EVENT_CLASS,...,2%,3.5%,5%,RESULT_CONV,RESULT_BEST,Delta2,Delta3.5,Delta5,Delta_Benchmark,PERF_SCALAR
0,16545,Zhou Xuanyu,9.12,DHS,13.5,10.0,C,Triple Jump,None,None,...,11.6424,11.46420,11.2860,9.12,9.12,-2.5224,-2.34420,-2.1660,-2.76,-18.232323
1,11022,"Zhong, Chuhan",12.26,None,None,None,None,Triple Jump,None,None,...,11.6424,11.46420,11.2860,12.26,12.26,0.6176,0.79580,0.9740,0.38,8.198653
2,1076,"Zheng, Justin De",10.47m,National Junior College,14,12,U15,Triple Jump,0,None,...,13.3574,13.15295,12.9485,10.47,10.47,-2.8874,-2.68295,-2.4785,-3.16,-18.184153
3,12853,Zheng Justin De,10.29,NJC,13.5,12.0,C,Triple Jump,None,None,...,13.3574,13.15295,12.9485,10.29,10.29,-3.0674,-2.86295,-2.6585,-3.34,-19.504769
4,2326,"Zhao, Daniel",11.67m,Hwa Chong Institution,14,3,U15,Triple Jump,0,None,...,13.3574,13.15295,12.9485,11.67,11.67,-1.6874,-1.48295,-1.2785,-1.96,-9.380044
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11356,3879,"., Shaik Isa",15.49,Club ZOOM,8,32,U9,Dash,80,,...,NaN,NaN,NaN,,NaN,NaN,NaN,NaN,NaN,NaN
11357,7899,"., Nur Elena",16.42,UNA,6,1,U7,Dash,80,,...,NaN,NaN,NaN,,NaN,NaN,NaN,NaN,NaN,NaN
11358,9907,"., Dharkshitha",08:35.6,Cedar Girls Secondary School,15,1,U18,Race Walk,1500,None,...,NaN,NaN,NaN,,NaN,NaN,NaN,NaN,NaN,NaN
11359,3520,"., Cayden",06:05.1,North Vista,12,54,Open,Mile Run,1,,...,NaN,NaN,NaN,,NaN,NaN,NaN,NaN,NaN,NaN


In [4153]:
# Identify Tier 1/2/3 performers

#top_performers_clean['TIER'] = np.where((top_performers_clean['Delta_Benchmark']>=0), 'Tier 1',    
#                                np.where(((top_performers_clean['Delta_Benchmark']<0) & (top_performers_clean['Delta2']>=0)), 'Tier2',
#                                np.where(((top_performers_clean['Delta2']<0) & (top_performers_clean['Delta3.5']>=0)), 'Tier3', ' ')))


tiered_performers['TIER'] = np.where((tiered_performers['Delta_Benchmark']>=0), 'Tier 1',    
                                np.where(((tiered_performers['Delta_Benchmark']<0) & (tiered_performers['Delta2']>=0)), 'Tier 2',
                                np.where(((tiered_performers['Delta2']<0) & (tiered_performers['Delta3.5']>=0)), 'Tier 3', 
                                np.where(((tiered_performers['Delta3.5']<0) & (tiered_performers['Delta5']>=0)), 'Tier 4', ' '))))



In [4154]:
# Drop rows without a SEAG benchmark

final_df = tiered_performers[tiered_performers['BENCHMARK'].notna()]


In [4155]:
final_df

,index,NAME,RESULT_x,TEAM,AGE,COMPETITION_RANK,DIVISION,EVENT_x,DISTANCE,EVENT_CLASS,...,3.5%,5%,RESULT_CONV,RESULT_BEST,Delta2,Delta3.5,Delta5,Delta_Benchmark,PERF_SCALAR,TIER
0,16545,Zhou Xuanyu,9.12,DHS,13.5,10.0,C,Triple Jump,None,None,...,11.46420,11.2860,9.12,9.12,-2.5224,-2.34420,-2.1660,-2.76,-18.232323,
1,11022,"Zhong, Chuhan",12.26,None,None,None,None,Triple Jump,None,None,...,11.46420,11.2860,12.26,12.26,0.6176,0.79580,0.9740,0.38,8.198653,Tier 1
2,1076,"Zheng, Justin De",10.47m,National Junior College,14,12,U15,Triple Jump,0,None,...,13.15295,12.9485,10.47,10.47,-2.8874,-2.68295,-2.4785,-3.16,-18.184153,
3,12853,Zheng Justin De,10.29,NJC,13.5,12.0,C,Triple Jump,None,None,...,13.15295,12.9485,10.29,10.29,-3.0674,-2.86295,-2.6585,-3.34,-19.504769,
4,2326,"Zhao, Daniel",11.67m,Hwa Chong Institution,14,3,U15,Triple Jump,0,None,...,13.15295,12.9485,11.67,11.67,-1.6874,-1.48295,-1.2785,-1.96,-9.380044,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11356,3879,"., Shaik Isa",15.49,Club ZOOM,8,32,U9,Dash,80,,...,NaN,NaN,,NaN,NaN,NaN,NaN,NaN,NaN,
11357,7899,"., Nur Elena",16.42,UNA,6,1,U7,Dash,80,,...,NaN,NaN,,NaN,NaN,NaN,NaN,NaN,NaN,
11358,9907,"., Dharkshitha",08:35.6,Cedar Girls Secondary School,15,1,U18,Race Walk,1500,None,...,NaN,NaN,,NaN,NaN,NaN,NaN,NaN,NaN,
11359,3520,"., Cayden",06:05.1,North Vista,12,54,Open,Mile Run,1,,...,NaN,NaN,,NaN,NaN,NaN,NaN,NaN,NaN,


In [4156]:
# Choose rows with mapped event

final_df=final_df[(final_df['MAPPED_EVENT']!=' ') & (final_df['TIER']!=' ')]

In [4157]:
os.chdir('/Users/veesheenyuen/Desktop/DataScience/SAA/Asian Youth/')


final_df.to_csv('AY_tiered_performers.csv', encoding='utf-8')

In [4158]:
final_df

,index,NAME,RESULT_x,TEAM,AGE,COMPETITION_RANK,DIVISION,EVENT_x,DISTANCE,EVENT_CLASS,...,3.5%,5%,RESULT_CONV,RESULT_BEST,Delta2,Delta3.5,Delta5,Delta_Benchmark,PERF_SCALAR,TIER
1,11022,"Zhong, Chuhan",12.26,None,None,None,None,Triple Jump,None,None,...,11.46420,11.2860,12.26,12.26,0.6176,0.79580,0.9740,0.38,8.198653,Tier 1
56,6870,"Tang Kai Sheng, Cayman",14.17m,VICTORIA SCHOOL,16,1,U18,Triple Jump,0,None,...,13.15295,12.9485,14.17,14.17,0.8126,1.01705,1.2215,0.54,8.961849,Tier 1
60,11299,"Tan, Tse Teng",11.77m,Hwa Chong Alumni Association,12,2,Open,Triple Jump,0,None,...,11.46420,11.2860,11.77,11.77,0.1276,0.30580,0.4840,-0.11,4.074074,Tier 2
78,9726,Tan Shou Yi Rei,14.99m,Raffles Institution JC,17,1,U20,Triple Jump,0,None,...,13.15295,12.9485,14.99,14.99,1.6326,1.83705,2.0415,1.36,14.977990,Tier 1
83,9476,"Sueto, Yuito",14.76m,Kansai University of Social We,12,3,Open,Triple Jump,0,None,...,13.15295,12.9485,14.76,14.76,1.4026,1.60705,1.8115,1.13,13.290536,Tier 1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8441,11066,"Boyd, Andre",11.16,Totality 360,12,7,Open,Dash,100,None,...,11.22975,11.3925,11.16,11.16,-0.0930,0.06975,0.2325,-0.31,2.142857,Tier 3
8447,1883,"Bin Zainal, Muhammad Fazrey",11.32,Singapore Institute of Technol,12,17,Open,Dash,100,,...,11.22975,11.3925,11.32,11.32,-0.2530,-0.09025,0.0725,-0.47,0.668203,Tier 4
8491,1370,"Ashok, Tharun",10.81,Ngee Ann Polytechnic,12,4,Open,Dash,100,,...,11.22975,11.3925,10.81,10.81,0.2570,0.41975,0.5825,0.04,5.368664,Tier 1
8540,3341,Aidan Michael Soh Zi Ren,11.16,INDIVIDUAL,18,None,None,100m,None,None,...,11.22975,11.3925,11.16,11.16,-0.0930,0.06975,0.2325,-0.31,2.142857,Tier 3


In [4159]:
#os.chdir('/Users/veesheenyuen/Desktop/DataScience/SAA/Asian Youth/')


#final_selection.to_csv('AY_final_selection.csv', encoding='utf-8')

In [4160]:
# Get list of names with no DOB

mask = ((final_df['DOB']=='None')|(final_df['DOB']=='nan'))

#mask = ((final_df['DOB'].isnull()))
#mask = final_df['DOB_new']=='NaT'

missing_names = final_df.loc[mask]

name_list = missing_names['NAME'].str.casefold().to_list()


In [4161]:
len(name_list)

117

In [4162]:
# Names with no DOB

missing_names

,index,NAME,RESULT_x,TEAM,AGE,COMPETITION_RANK,DIVISION,EVENT_x,DISTANCE,EVENT_CLASS,...,3.5%,5%,RESULT_CONV,RESULT_BEST,Delta2,Delta3.5,Delta5,Delta_Benchmark,PERF_SCALAR,TIER
99,804,"Saw, Xiang Yu",13.92m,Singapore Institute of Managem,12,2,Open,Triple Jump,0,None,...,13.15295,12.9485,13.92,13.92,0.5626,0.76705,0.9715,0.29,7.127660,Tier 1
195,17015,"Lau, Jia Hern",13.63,RI,18.5,3.0,A,Triple Jump,None,None,...,13.15295,12.9485,13.63,13.63,0.2726,0.47705,0.6815,0.00,5.000000,Tier 1
242,17016,"Fu, Song",13.52,EJC,18.5,4.0,A,Triple Jump,None,None,...,13.15295,12.9485,13.52,13.52,0.1626,0.36705,0.5715,-0.11,4.192957,Tier 2
275,17018,Chang I Teng,13.01,DHS,18.5,6.0,A,Triple Jump,None,None,...,13.15295,12.9485,13.01,13.01,-0.3474,-0.14295,0.0615,-0.62,0.451211,Tier 4
347,14932,"Ng, Gabriel",16.49,RI,18.5,2.0,A,Shot Put,None,5.00kg,...,16.10585,15.8555,16.49,16.49,0.1338,0.38415,0.6345,-0.20,3.801678,Tier 2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8290,17173,"Chua, Garrett",00:10.90,RI,18.5,2.0,A,100m,None,None,...,11.22975,11.3925,10.9,10.90,0.1670,0.32975,0.4925,-0.05,4.539171,Tier 2
8327,15836,Chloe Sarahena Tan Hui En,00:12.85,EJC,18.5,6.0,A,100m,None,None,...,12.67875,12.8625,12.85,12.85,-0.3550,-0.17125,0.0125,-0.60,0.102041,Tier 4
8328,17152,Chloe Chee En-Ya,00:12.44,ACS(I),18.5,4.0,A,100m,None,None,...,12.67875,12.8625,12.44,12.44,0.0550,0.23875,0.4225,-0.19,3.448980,Tier 2
8433,10792,Bryan Ng,10.93,None,None,2,None,100m,None,None,...,11.22975,11.3925,10.93,10.93,0.1370,0.29975,0.4625,-0.08,4.262673,Tier 2


In [4163]:
final_df['DOB_dt']

1       2005-10-18 00:00:00
56      2009-07-15 00:00:00
60      2002-11-28 00:00:00
78      2008-12-05 00:00:00
83      2004-01-06 00:00:00
               ...         
8441    2005-10-27 00:00:00
8447    1997-09-04 00:00:00
8491    2005-07-02 00:00:00
8540    2007-06-19 00:00:00
8552    2007-07-06 00:00:00
Name: DOB_dt, Length: 445, dtype: object

In [4165]:
import string

# Use clean name without punctuation and dictionary_dob_clean_name to lookup the rest
# Create dictionary version with punctuation stripped out

translator = str.maketrans('', '', string.punctuation)
final_df['clean_name'] = final_df['NAME'].str.translate(translator)
final_df['clean_name'] = final_df['clean_name'].str.casefold()

final_df

/var/folders/q5/yf8g5p896_b94gkbhqcjx3t40000gn/T/ipykernel_51640/2491666764.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_df['clean_name'] = final_df['NAME'].str.translate(translator)
/var/folders/q5/yf8g5p896_b94gkbhqcjx3t40000gn/T/ipykernel_51640/2491666764.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_df['clean_name'] = final_df['clean_name'].str.casefold()


,index,NAME,RESULT_x,TEAM,AGE,COMPETITION_RANK,DIVISION,EVENT_x,DISTANCE,EVENT_CLASS,...,5%,RESULT_CONV,RESULT_BEST,Delta2,Delta3.5,Delta5,Delta_Benchmark,PERF_SCALAR,TIER,clean_name
1,11022,"Zhong, Chuhan",12.26,None,None,None,None,Triple Jump,None,None,...,11.2860,12.26,12.26,0.6176,0.79580,0.9740,0.38,8.198653,Tier 1,zhong chuhan
56,6870,"Tang Kai Sheng, Cayman",14.17m,VICTORIA SCHOOL,16,1,U18,Triple Jump,0,None,...,12.9485,14.17,14.17,0.8126,1.01705,1.2215,0.54,8.961849,Tier 1,tang kai sheng cayman
60,11299,"Tan, Tse Teng",11.77m,Hwa Chong Alumni Association,12,2,Open,Triple Jump,0,None,...,11.2860,11.77,11.77,0.1276,0.30580,0.4840,-0.11,4.074074,Tier 2,tan tse teng
78,9726,Tan Shou Yi Rei,14.99m,Raffles Institution JC,17,1,U20,Triple Jump,0,None,...,12.9485,14.99,14.99,1.6326,1.83705,2.0415,1.36,14.977990,Tier 1,tan shou yi rei
83,9476,"Sueto, Yuito",14.76m,Kansai University of Social We,12,3,Open,Triple Jump,0,None,...,12.9485,14.76,14.76,1.4026,1.60705,1.8115,1.13,13.290536,Tier 1,sueto yuito
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8441,11066,"Boyd, Andre",11.16,Totality 360,12,7,Open,Dash,100,None,...,11.3925,11.16,11.16,-0.0930,0.06975,0.2325,-0.31,2.142857,Tier 3,boyd andre
8447,1883,"Bin Zainal, Muhammad Fazrey",11.32,Singapore Institute of Technol,12,17,Open,Dash,100,,...,11.3925,11.32,11.32,-0.2530,-0.09025,0.0725,-0.47,0.668203,Tier 4,bin zainal muhammad fazrey
8491,1370,"Ashok, Tharun",10.81,Ngee Ann Polytechnic,12,4,Open,Dash,100,,...,11.3925,10.81,10.81,0.2570,0.41975,0.5825,0.04,5.368664,Tier 1,ashok tharun
8540,3341,Aidan Michael Soh Zi Ren,11.16,INDIVIDUAL,18,None,None,100m,None,None,...,11.3925,11.16,11.16,-0.0930,0.06975,0.2325,-0.31,2.142857,Tier 3,aidan michael soh zi ren


In [4166]:
# Split name elements into a list

final_df['name_to_list'] = final_df['clean_name'].str.split(' ')

/var/folders/q5/yf8g5p896_b94gkbhqcjx3t40000gn/T/ipykernel_51640/577288367.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_df['name_to_list'] = final_df['clean_name'].str.split(' ')


In [4167]:
final_df = final_df.reset_index(drop=True)


In [4168]:
final_df

,index,NAME,RESULT_x,TEAM,AGE,COMPETITION_RANK,DIVISION,EVENT_x,DISTANCE,EVENT_CLASS,...,RESULT_CONV,RESULT_BEST,Delta2,Delta3.5,Delta5,Delta_Benchmark,PERF_SCALAR,TIER,clean_name,name_to_list
0,11022,"Zhong, Chuhan",12.26,None,None,None,None,Triple Jump,None,None,...,12.26,12.26,0.6176,0.79580,0.9740,0.38,8.198653,Tier 1,zhong chuhan,"[zhong, chuhan]"
1,6870,"Tang Kai Sheng, Cayman",14.17m,VICTORIA SCHOOL,16,1,U18,Triple Jump,0,None,...,14.17,14.17,0.8126,1.01705,1.2215,0.54,8.961849,Tier 1,tang kai sheng cayman,"[tang, kai, sheng, cayman]"
2,11299,"Tan, Tse Teng",11.77m,Hwa Chong Alumni Association,12,2,Open,Triple Jump,0,None,...,11.77,11.77,0.1276,0.30580,0.4840,-0.11,4.074074,Tier 2,tan tse teng,"[tan, tse, teng]"
3,9726,Tan Shou Yi Rei,14.99m,Raffles Institution JC,17,1,U20,Triple Jump,0,None,...,14.99,14.99,1.6326,1.83705,2.0415,1.36,14.977990,Tier 1,tan shou yi rei,"[tan, shou, yi, rei]"
4,9476,"Sueto, Yuito",14.76m,Kansai University of Social We,12,3,Open,Triple Jump,0,None,...,14.76,14.76,1.4026,1.60705,1.8115,1.13,13.290536,Tier 1,sueto yuito,"[sueto, yuito]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
440,11066,"Boyd, Andre",11.16,Totality 360,12,7,Open,Dash,100,None,...,11.16,11.16,-0.0930,0.06975,0.2325,-0.31,2.142857,Tier 3,boyd andre,"[boyd, andre]"
441,1883,"Bin Zainal, Muhammad Fazrey",11.32,Singapore Institute of Technol,12,17,Open,Dash,100,,...,11.32,11.32,-0.2530,-0.09025,0.0725,-0.47,0.668203,Tier 4,bin zainal muhammad fazrey,"[bin, zainal, muhammad, fazrey]"
442,1370,"Ashok, Tharun",10.81,Ngee Ann Polytechnic,12,4,Open,Dash,100,,...,10.81,10.81,0.2570,0.41975,0.5825,0.04,5.368664,Tier 1,ashok tharun,"[ashok, tharun]"
443,3341,Aidan Michael Soh Zi Ren,11.16,INDIVIDUAL,18,None,None,100m,None,None,...,11.16,11.16,-0.0930,0.06975,0.2325,-0.31,2.142857,Tier 3,aidan michael soh zi ren,"[aidan, michael, soh, zi, ren]"


In [4183]:
# Determine all permutations of name

for index, row in final_df.iterrows():
         
   # if str(row['DOB_new']) == 'NaT':
   # if ((final_df['DOB']=='None') or (final_df['DOB']=='nan')):
            
    name_components = row['name_to_list']
        
    permutations_list = []
    dob_list=[]
    string=''
    
    if row['DOB_dt']=='':  # filter out those DOB with only one digit
        
        for p in permutations(name_components):
        
            string = " ".join(p)
        
            permutations_list.append(string)

            dob = dictionary_dob_clean_name.get(string)
        
            if dob is not None and len(str(dob))>2:
                dob_list.append(dob)
            
       #     final_df.at[index, 'permutations'] = permutations_list
            final_df.at[index, 'dob_list'] = dob_list
        
        
        if len(dob_list)==1:
            final_df.loc[index, 'dob_mapped'] = dob_list[0]
        elif len(dob_list)==2:
            final_df.loc[index, 'dob_mapped'] = dob_list[0]
            
   # print(name, dictionary_dob_clean_name.get(name))  
        
    #null.loc[index, 'DOB_found'] = dictionary_dob_clean_name.get(name)


In [4184]:
final_df

,index,NAME,RESULT_x,TEAM,AGE,COMPETITION_RANK,DIVISION,EVENT_x,DISTANCE,EVENT_CLASS,...,Delta5,Delta_Benchmark,PERF_SCALAR,TIER,clean_name,name_to_list,permutations,dob_mapped,DOB_final,dob_list
0,11022,"Zhong, Chuhan",12.26,None,None,None,None,Triple Jump,None,None,...,0.9740,0.38,8.198653,Tier 1,zhong chuhan,"[zhong, chuhan]",nan,nan,2005-10-18 00:00:00,nan
1,6870,"Tang Kai Sheng, Cayman",14.17m,VICTORIA SCHOOL,16,1,U18,Triple Jump,0,None,...,1.2215,0.54,8.961849,Tier 1,tang kai sheng cayman,"[tang, kai, sheng, cayman]",nan,nan,2009-07-15 00:00:00,nan
2,11299,"Tan, Tse Teng",11.77m,Hwa Chong Alumni Association,12,2,Open,Triple Jump,0,None,...,0.4840,-0.11,4.074074,Tier 2,tan tse teng,"[tan, tse, teng]",nan,nan,2002-11-28 00:00:00,nan
3,9726,Tan Shou Yi Rei,14.99m,Raffles Institution JC,17,1,U20,Triple Jump,0,None,...,2.0415,1.36,14.977990,Tier 1,tan shou yi rei,"[tan, shou, yi, rei]",nan,nan,2008-12-05 00:00:00,nan
4,9476,"Sueto, Yuito",14.76m,Kansai University of Social We,12,3,Open,Triple Jump,0,None,...,1.8115,1.13,13.290536,Tier 1,sueto yuito,"[sueto, yuito]",nan,nan,2004-01-06 00:00:00,nan
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
440,11066,"Boyd, Andre",11.16,Totality 360,12,7,Open,Dash,100,None,...,0.2325,-0.31,2.142857,Tier 3,boyd andre,"[boyd, andre]",nan,nan,2005-10-27 00:00:00,nan
441,1883,"Bin Zainal, Muhammad Fazrey",11.32,Singapore Institute of Technol,12,17,Open,Dash,100,,...,0.0725,-0.47,0.668203,Tier 4,bin zainal muhammad fazrey,"[bin, zainal, muhammad, fazrey]",nan,nan,1997-09-04 00:00:00,nan
442,1370,"Ashok, Tharun",10.81,Ngee Ann Polytechnic,12,4,Open,Dash,100,,...,0.5825,0.04,5.368664,Tier 1,ashok tharun,"[ashok, tharun]",nan,nan,2005-07-02 00:00:00,nan
443,3341,Aidan Michael Soh Zi Ren,11.16,INDIVIDUAL,18,None,None,100m,None,None,...,0.2325,-0.31,2.142857,Tier 3,aidan michael soh zi ren,"[aidan, michael, soh, zi, ren]",nan,nan,2007-06-19 00:00:00,nan


In [4185]:
# Consolidate all the DOB fields into one column

condition1 = final_df['DOB_dt']!=''
#condition2=((df['CATEGORY_EVENT']=='Jump')|(df['CATEGORY_EVENT']=='Throw'))
#condition3=df['SEED_CONV']<df['RESULT_CONV']
#condition4=~((df['CATEGORY_EVENT']=='Jump')|(df['CATEGORY_EVENT']=='Throw'))


final_df['DOB_final'] = final_df['DOB_dt'].where((condition1), final_df['dob_mapped'].values)


In [4186]:
os.chdir('/Users/veesheenyuen/Desktop/DataScience/SAA/Asian Youth/')


final_df.to_csv('final_df.csv', encoding='utf-8')

In [4193]:
final_df['DOB_final'] = pd.to_datetime(final_df['DOB_final'])


birth_start = datetime.datetime(2009, 1, 1)
birth_end = datetime.datetime(2010, 12, 31)

start_date = np.datetime64(birth_start)
end_date = np.datetime64(birth_end)


mask = (final_df['DOB_final'] >= start_date) & (final_df['DOB_final'] <= end_date)
final_selection = final_df.loc[mask]


In [4194]:
final_selection

,index,NAME,RESULT_x,TEAM,AGE,COMPETITION_RANK,DIVISION,EVENT_x,DISTANCE,EVENT_CLASS,...,PERF_SCALAR,TIER,clean_name,name_to_list,permutations,dob_mapped,DOB_final,dob_list,year_extract,age_extract
1,6870,"Tang Kai Sheng, Cayman",14.17m,VICTORIA SCHOOL,16,1,U18,Triple Jump,0,None,...,8.961849,Tier 1,tang kai sheng cayman,"[tang, kai, sheng, cayman]",nan,NaT,2009-07-15,nan,NaN,NaN
36,4533,"Jiang, Yunfan",4.03m,VICTORIA SCHOOL,16,1,U18,Pole Vault,0,None,...,5.000000,Tier 1,jiang yunfan,"[jiang, yunfan]",nan,NaT,2009-07-20,nan,NaN,NaN
66,13907,"Lam, Kristel",34.24,CG,16,1.0,B,Javelin,None,500g,...,6.904762,Tier 1,lam kristel,"[lam, kristel]",nan,2010-12-22,2010-12-22,[2010-12-22 00:00:00],2010.0,15.0
91,11411,"Lee, Yu Foong",48.29m,Hwa Chong Institution,18.5,2,A Div,Discus Throw,0,(1.5kg),...,1.696035,Tier 3,lee yu foong,"[lee, yu, foong]",nan,NaT,2009-03-16,nan,NaN,NaN
105,5804,"Tan Yu En, Bernice",02:28.9,TeamFabian,12,3,Open,Run,800,,...,1.026814,Tier 4,tan yu en bernice,"[tan, yu, en, bernice]",nan,NaT,2010-10-05,nan,NaN,NaN
112,16307,Saicharan Ganeshkumar,02:04.77,VS,16,3.0,B,800m,None,None,...,1.128871,Tier 4,saicharan ganeshkumar,"[saicharan, ganeshkumar]",nan,2009-03-06,2009-03-06,[2009-03-06 00:00:00],2009.0,16.0
123,12266,"Loo, Javen Loo Yan Kai",02:04.2,Bukit View Secondary School,12,5,Open,Run,800,None,...,1.603397,Tier 3,loo javen loo yan kai,"[loo, javen, loo, yan, kai]",nan,NaT,2010-07-11,nan,NaN,NaN
128,16305,"Kuah Yong Chuan, Ezekiel",02:03.64,SST,16,1.0,B,800m,None,None,...,2.069597,Tier 3,kuah yong chuan ezekiel,"[kuah, yong, chuan, ezekiel]",nan,2009-02-12,2009-02-12,[2009-02-12 00:00:00],2009.0,16.0
133,16289,"Jeffrey, Deianira",02:29.53,MGS,16,1.0,B,800m,None,None,...,0.586900,Tier 4,jeffrey deianira,"[jeffrey, deianira]",nan,2010-09-30,2010-09-30,[2010-09-30 00:00:00],2010.0,15.0
137,16306,"Hayagreev, Jeyaseelan Sai",02:04.34,BTV,16,2.0,B,800m,None,None,...,1.486846,Tier 4,hayagreev jeyaseelan sai,"[hayagreev, jeyaseelan, sai]",nan,2009-09-24,2009-09-24,[2009-09-24 00:00:00],2009.0,16.0


In [4188]:
final_df['dob_mapped'] = pd.to_datetime(final_df['dob_mapped'], errors='coerce')

final_df['year_extract'] = final_df['dob_mapped'].dt.strftime('%Y')

final_df['year_extract'] = pd.to_numeric(final_df['year_extract'])

final_df['age_extract'] = 2025 - final_df['year_extract']


In [4189]:
def age(number):  # correct negative age numbers
    
    if number<0:
        
        number+=100
        
    return number


final_df['age_extract']=final_df['age_extract'].apply(age)


In [2570]:
# If NSG event then choose AGE otherwise choose age_extract

condition1 = final_df['COMPETITION']=='National School Games'

final_df['age_extract'] = final_df['AGE'].where((condition1), final_df['age_extract'].values)
final_df[['age_extract']] = final_df[['age_extract']].apply(pd.to_numeric)

/var/folders/q5/yf8g5p896_b94gkbhqcjx3t40000gn/T/ipykernel_51640/1013484073.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_df['age_extract'] = final_df['AGE'].where((condition1), final_df['age_extract'].values)
/var/folders/q5/yf8g5p896_b94gkbhqcjx3t40000gn/T/ipykernel_51640/1013484073.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_df[['age_extract']] = final_df[['age_extract']].apply(pd.to_numeric)


In [2573]:
final_df

,index,NAME,RESULT_x,TEAM,AGE,COMPETITION_RANK,DIVISION,EVENT_x,DISTANCE,EVENT_CLASS,...,3.5%,5%,RESULT_CONV,RESULT_BEST,Delta2,Delta3.5,Delta5,Delta_Benchmark,PERF_SCALAR,TIER
1,11022,"Zhong, Chuhan",12.26,None,None,None,None,Triple Jump,None,None,...,11.46420,11.2860,12.26,12.26,0.6176,0.79580,0.9740,0.38,8.198653,Tier 1
47,12316,Tse Teng Tan,11.59,None,None,None,None,Triple Jump,None,None,...,11.46420,11.2860,11.59,11.59,-0.0524,0.12580,0.3040,-0.29,2.558923,Tier 3
57,6870,"Tang Kai Sheng, Cayman",14.17m,VICTORIA SCHOOL,16,1,U18,Triple Jump,0,None,...,13.15295,12.9485,14.17,14.17,0.8126,1.01705,1.2215,0.54,8.961849,Tier 1
61,11299,"Tan, Tse Teng",11.77m,Hwa Chong Alumni Association,12,2,Open,Triple Jump,0,None,...,11.46420,11.2860,11.77,11.77,0.1276,0.30580,0.4840,-0.11,4.074074,Tier 2
79,9726,Tan Shou Yi Rei (Chen Shouyi),14.99m,Raffles Institution JC,17,1,U20,Triple Jump,0,None,...,13.15295,12.9485,14.99,14.99,1.6326,1.83705,2.0415,1.36,14.977990,Tier 1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8462,1883,"Bin Zainal, Muhammad Fazrey",11.32,Singapore Institute of Technol,12,17,Open,Dash,100,,...,11.22975,11.3925,11.32,11.32,-0.2530,-0.09025,0.0725,-0.47,0.668203,Tier 4
8506,1370,"Ashok, Tharun",10.81,Ngee Ann Polytechnic,12,4,Open,Dash,100,,...,11.22975,11.3925,10.81,10.81,0.2570,0.41975,0.5825,0.04,5.368664,Tier 1
8533,889,"Ang, Jamie El-Red",11.09,Oldham Athletics,12,19,Open,Dash,100,None,...,11.22975,11.3925,11.09,11.09,-0.0230,0.13975,0.3025,-0.24,2.788018,Tier 3
8556,3341,Aidan Michael Soh,11.16,INDIVIDUAL,18,None,None,100m,None,None,...,11.22975,11.3925,11.16,11.16,-0.0930,0.06975,0.2325,-0.31,2.142857,Tier 3


In [2574]:
os.chdir('/Users/veesheenyuen/Desktop/DataScience/SAA/Asian Youth/')


final_df.to_csv('final_df.csv', encoding='utf-8')

In [2578]:
# Filter out nan DOB first and list selection.  We deal with nan DOB next step

selection_with_dob = final_df[(final_df['DOB'])!='None']

selection_with_dob

,index,NAME,RESULT_x,TEAM,AGE,COMPETITION_RANK,DIVISION,EVENT_x,DISTANCE,EVENT_CLASS,...,3.5%,5%,RESULT_CONV,RESULT_BEST,Delta2,Delta3.5,Delta5,Delta_Benchmark,PERF_SCALAR,TIER
1,11022,"Zhong, Chuhan",12.26,None,None,None,None,Triple Jump,None,None,...,11.46420,11.2860,12.26,12.26,0.6176,0.79580,0.9740,0.38,8.198653,Tier 1
57,6870,"Tang Kai Sheng, Cayman",14.17m,VICTORIA SCHOOL,16,1,U18,Triple Jump,0,None,...,13.15295,12.9485,14.17,14.17,0.8126,1.01705,1.2215,0.54,8.961849,Tier 1
61,11299,"Tan, Tse Teng",11.77m,Hwa Chong Alumni Association,12,2,Open,Triple Jump,0,None,...,11.46420,11.2860,11.77,11.77,0.1276,0.30580,0.4840,-0.11,4.074074,Tier 2
79,9726,Tan Shou Yi Rei (Chen Shouyi),14.99m,Raffles Institution JC,17,1,U20,Triple Jump,0,None,...,13.15295,12.9485,14.99,14.99,1.6326,1.83705,2.0415,1.36,14.977990,Tier 1
84,9476,"Sueto, Yuito",14.76m,Kansai University of Social We,12,3,Open,Triple Jump,0,None,...,13.15295,12.9485,14.76,14.76,1.4026,1.60705,1.8115,1.13,13.290536,Tier 1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8462,1883,"Bin Zainal, Muhammad Fazrey",11.32,Singapore Institute of Technol,12,17,Open,Dash,100,,...,11.22975,11.3925,11.32,11.32,-0.2530,-0.09025,0.0725,-0.47,0.668203,Tier 4
8506,1370,"Ashok, Tharun",10.81,Ngee Ann Polytechnic,12,4,Open,Dash,100,,...,11.22975,11.3925,10.81,10.81,0.2570,0.41975,0.5825,0.04,5.368664,Tier 1
8533,889,"Ang, Jamie El-Red",11.09,Oldham Athletics,12,19,Open,Dash,100,None,...,11.22975,11.3925,11.09,11.09,-0.0230,0.13975,0.3025,-0.24,2.788018,Tier 3
8556,3341,Aidan Michael Soh,11.16,INDIVIDUAL,18,None,None,100m,None,None,...,11.22975,11.3925,11.16,11.16,-0.0930,0.06975,0.2325,-0.31,2.142857,Tier 3


In [ ]:

selection_with_dob[['age_extract']] = selection_with_dob[['age_extract']].apply(pd.to_numeric)

mask = (((selection_with_dob['age_extract'] >= 14) & (selection_with_dob['age_extract'] <= 16))|(selection_with_dob['age_extract'].isna()))
#mask = ((final_df['age_extract'].isnull()))


selection_with_dob = selection_with_dob.loc[mask]


In [2572]:
selection_with_dob

,index,NAME,RESULT_x,TEAM,AGE,COMPETITION_RANK,DIVISION,EVENT_x,DISTANCE,EVENT_CLASS,...,3.5%,5%,RESULT_CONV,RESULT_BEST,Delta2,Delta3.5,Delta5,Delta_Benchmark,PERF_SCALAR,TIER
1,11022,"Zhong, Chuhan",12.26,None,None,None,None,Triple Jump,None,None,...,11.46420,11.2860,12.26,12.26,0.6176,0.79580,0.9740,0.38,8.198653,Tier 1
47,12316,Tse Teng Tan,11.59,None,None,None,None,Triple Jump,None,None,...,11.46420,11.2860,11.59,11.59,-0.0524,0.12580,0.3040,-0.29,2.558923,Tier 3
57,6870,"Tang Kai Sheng, Cayman",14.17m,VICTORIA SCHOOL,16,1,U18,Triple Jump,0,None,...,13.15295,12.9485,14.17,14.17,0.8126,1.01705,1.2215,0.54,8.961849,Tier 1
61,11299,"Tan, Tse Teng",11.77m,Hwa Chong Alumni Association,12,2,Open,Triple Jump,0,None,...,11.46420,11.2860,11.77,11.77,0.1276,0.30580,0.4840,-0.11,4.074074,Tier 2
84,9476,"Sueto, Yuito",14.76m,Kansai University of Social We,12,3,Open,Triple Jump,0,None,...,13.15295,12.9485,14.76,14.76,1.4026,1.60705,1.8115,1.13,13.290536,Tier 1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8448,10792,Bryan Ng,10.93,None,None,2,None,100m,None,None,...,11.22975,11.3925,10.93,10.93,0.1370,0.29975,0.4625,-0.08,4.262673,Tier 2
8462,1883,"Bin Zainal, Muhammad Fazrey",11.32,Singapore Institute of Technol,12,17,Open,Dash,100,,...,11.22975,11.3925,11.32,11.32,-0.2530,-0.09025,0.0725,-0.47,0.668203,Tier 4
8506,1370,"Ashok, Tharun",10.81,Ngee Ann Polytechnic,12,4,Open,Dash,100,,...,11.22975,11.3925,10.81,10.81,0.2570,0.41975,0.5825,0.04,5.368664,Tier 1
8533,889,"Ang, Jamie El-Red",11.09,Oldham Athletics,12,19,Open,Dash,100,None,...,11.22975,11.3925,11.09,11.09,-0.0230,0.13975,0.3025,-0.24,2.788018,Tier 3
